# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
import imageio
from PIL import Image
from skimage.transform import resize
# from skimage.transform import resize
# from matplotlib.pyplot import imread
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('C:\\Users\\Amit\\Documents\\Academics\\upGrad\\Deep Learning\\Project_data\\train.csv').readlines())
val_doc = np.random.permutation(open('C:\\Users\\Amit\\Documents\\Academics\\upGrad\\Deep Learning\\Project_data\\val.csv').readlines())
batch_size = 32 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
# source_path = train_path i.e. path of 663 folders
# folder_list = train_doc i.e. csv file
# batch_size = 64

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    # It is not possible to work with all the 30 images, as it will take too long processing time.
    # So lets choose randomly 18 images
    img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29] #create a list of image numbers you want to use for a particular video(incase if u want to try with lesser images)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,18,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image[:,20:140,:],(84,84)).astype(np.float32)
                                     
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255.0 #normalise and feed in the image # divide by 255.0
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255.0 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size)!=0:
            batch_data = np.zeros((len(t)%batch_size,18,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image[:,20:140,:],(84,84)).astype(np.float32)
                                      
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255.0 #normalise and feed in the image # divide by 255.0
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255.0 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255.0 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels# source_path = train_path i.e. path of 663 folders

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [4]:
curr_dt_time = datetime.datetime.now()
train_path = 'C:\\Users\\Amit\\Documents\\Academics\\upGrad\\Deep Learning\\Project_data\\train'
val_path = 'C:\\Users\\Amit\\Documents\\Academics\\upGrad\\Deep Learning\\Project_data\\val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model_1 = Sequential()

model_1.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_1.add(BatchNormalization())
model_1.add(Activation('elu'))
model_1.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))


model_1.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_1.add(BatchNormalization())
model_1.add(Activation('elu'))
model_1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model_1.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_1.add(BatchNormalization())
model_1.add(Activation('elu'))
model_1.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))


model_1.add(Flatten())
model_1.add(Dropout(0.5))
model_1.add(Dense(512, activation='elu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

optimiser = keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation (Activation)      (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 42, 84, 64)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 9, 42, 84, 128)    221312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 42, 84, 128)    512       
_________________________________________________________________
activation_1 (Activation)    (None, 9, 42, 84, 128)    0

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [11]:
# Using model_1.fit instead of fit_generator (as the latter has been deprecated):

model_1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:\Users\Amit\Documents\Academics\upGrad\Deep Learning\Project_data\train ; batch size = 32
Epoch 1/50
21/21 [==============================] - 69s 3s/step - loss: 7.8376 - categorical_accuracy: 0.2941 - val_loss: 3.0298 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model_init_2021-10-2714_06_11.512848\model-00001-7.83759-0.29412-3.02976-0.18000.h5
Epoch 2/50
21/21 [==============================] - 52s 3s/step - loss: 1.7301 - categorical_accuracy: 0.4012 - val_loss: 2.4565 - val_categorical_accuracy: 0.1800

Epoch 00002: saving model to model_init_2021-10-2714_06_11.512848\model-00002-1.73008-0.40121-2.45645-0.18000.h5
Epoch 3/50
21/21 [==============================] - 52s 3s/step - loss: 1.5317 - categorical_accuracy: 0.4827 - val_loss: 4.5129 - val_categorical_accuracy: 0.1400

Epoch 00003: saving model to model_init_2021-10-2714_06_11.512848\model-00003-1.53171-0.48265-4.51288-0.14000.h5
Epoch 4/50
21/21 [==============================] - 50s 2s/

In [ ]:
# Model 1 appears to be slightly overfitting. But the overall performance appears to be good.

In [12]:
#write your model here
model_2 = Sequential()

model_2.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(2,2,2)))

model_2.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(2,2,2)))

model_2.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_2.add(BatchNormalization())
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(2,2,2)))

model_2.add(Flatten())
model_2.add(Dropout(0.5))
model_2.add(Dense(1024, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(5, activation='softmax'))

In [13]:
model_2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 18, 84, 84, 32)    2624      
_________________________________________________________________
batch_normalization_3 (Batch (None, 18, 84, 84, 32)    128       
_________________________________________________________________
activation_3 (Activation)    (None, 18, 84, 84, 32)    0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 18, 84, 84, 64)    55360     
_________________________________________________________________
batch_normalization_4 (Batch (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation_4 (Activation)    (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 9, 42, 42, 64)    

In [14]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [15]:
model_2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 61s 3s/step - loss: 4.1823 - categorical_accuracy: 0.2142 - val_loss: 1.6172 - val_categorical_accuracy: 0.1500

Epoch 00001: val_loss improved from inf to 1.61719, saving model to model_init_2021-10-2714_06_11.512848\model-00001-4.18232-0.21418-1.61719-0.15000.h5
Epoch 2/50
21/21 [==============================] - 50s 2s/step - loss: 4.1129 - categorical_accuracy: 0.2051 - val_loss: 1.6210 - val_categorical_accuracy: 0.1900

Epoch 00002: val_loss did not improve from 1.61719
Epoch 3/50
21/21 [==============================] - 52s 3s/step - loss: 3.8472 - categorical_accuracy: 0.2217 - val_loss: 1.6334 - val_categorical_accuracy: 0.1600

Epoch 00003: val_loss did not improve from 1.61719
Epoch 4/50
21/21 [==============================] - 52s 3s/step - loss: 3.9244 - categorical_accuracy: 0.2353 - val_loss: 1.6662 - val_categorical_accuracy: 0.1100

Epoch 00004: val_loss did not improve from 1.61719
Epoch 5/50
21/21 [=================

In [ ]:
# Model 2 is underfitting & overall performance is poor

In [17]:
#write your model here
model_3 = Sequential()

model_3.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_3.add(BatchNormalization())
model_3.add(Activation('relu'))
model_3.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_3.add(BatchNormalization())
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(2,2,2)))

model_3.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_3.add(BatchNormalization())
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(2,2,2)))

model_3.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_3.add(BatchNormalization())
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(2,2,2)))

model_3.add(Flatten())
model_3.add(Dropout(0.5))
model_3.add(Dense(1024, activation='relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(512, activation='relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(5, activation='softmax'))

In [18]:
model_3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_3.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_7 (Conv3D)            (None, 18, 84, 84, 32)    2624      
_________________________________________________________________
batch_normalization_7 (Batch (None, 18, 84, 84, 32)    128       
_________________________________________________________________
activation_7 (Activation)    (None, 18, 84, 84, 32)    0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 18, 84, 84, 64)    55360     
_________________________________________________________________
batch_normalization_8 (Batch (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation_8 (Activation)    (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 9, 42, 42, 64)    

In [19]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [20]:
model_3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 52s 3s/step - loss: 4.3237 - categorical_accuracy: 0.2142 - val_loss: 1.6140 - val_categorical_accuracy: 0.1800

Epoch 00001: val_loss improved from inf to 1.61401, saving model to model_init_2021-10-2714_06_11.512848\model-00001-4.32365-0.21418-1.61401-0.18000.h5
Epoch 2/50
21/21 [==============================] - 51s 3s/step - loss: 4.0262 - categorical_accuracy: 0.1946 - val_loss: 1.6250 - val_categorical_accuracy: 0.1700

Epoch 00002: val_loss did not improve from 1.61401
Epoch 3/50
21/21 [==============================] - 53s 3s/step - loss: 3.9014 - categorical_accuracy: 0.2323 - val_loss: 1.6117 - val_categorical_accuracy: 0.2000

Epoch 00003: val_loss improved from 1.61401 to 1.61174, saving model to model_init_2021-10-2714_06_11.512848\model-00003-3.90136-0.23228-1.61174-0.20000.h5
Epoch 4/50
21/21 [==============================] - 50s 2s/step - loss: 3.7846 - categorical_accuracy: 0.2142 - val_loss: 1.6247 - val_categorical

21/21 [==============================] - 52s 3s/step - loss: 2.2028 - categorical_accuracy: 0.3077 - val_loss: 1.2727 - val_categorical_accuracy: 0.5000

Epoch 00033: val_loss did not improve from 1.26036
Epoch 34/50
21/21 [==============================] - 51s 3s/step - loss: 2.0663 - categorical_accuracy: 0.3017 - val_loss: 1.2627 - val_categorical_accuracy: 0.4800

Epoch 00034: val_loss did not improve from 1.26036
Epoch 35/50
21/21 [==============================] - 51s 3s/step - loss: 2.1987 - categorical_accuracy: 0.2911 - val_loss: 1.2368 - val_categorical_accuracy: 0.4900

Epoch 00035: val_loss improved from 1.26036 to 1.23677, saving model to model_init_2021-10-2714_06_11.512848\model-00035-2.19868-0.29110-1.23677-0.49000.h5
Epoch 36/50
21/21 [==============================] - 49s 2s/step - loss: 2.0829 - categorical_accuracy: 0.3363 - val_loss: 1.2787 - val_categorical_accuracy: 0.5000

Epoch 00036: val_loss did not improve from 1.23677
Epoch 37/50
21/21 [====================

In [ ]:
# Model 3 is underfitting and overall performance is poor

In [ ]:
# Trying model 1 with 50% dropouts

In [21]:
#write your model here
model_4 = Sequential()

model_4.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_4.add(BatchNormalization())
model_4.add(Activation('elu'))
model_4.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_4.add(BatchNormalization())
model_4.add(Activation('elu'))
model_4.add(MaxPooling3D(pool_size=(2,2,1)))
model_4.add(Dropout(0.25))

model_4.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_4.add(BatchNormalization())
model_4.add(Activation('elu'))
model_4.add(MaxPooling3D(pool_size=(2,2,2)))
model_4.add(Dropout(0.25))

model_4.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_4.add(BatchNormalization())
model_4.add(Activation('elu'))
model_4.add(MaxPooling3D(pool_size=(2,2,1)))
model_4.add(Dropout(0.25))

model_4.add(Flatten())
model_4.add(Dropout(0.5))
model_4.add(Dense(1024, activation='elu'))
model_4.add(Dropout(0.5))
model_4.add(Dense(512, activation='elu'))
model_4.add(Dropout(0.5))
model_4.add(Dense(5, activation='softmax'))

In [22]:
model_4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_4.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_11 (Conv3D)           (None, 18, 84, 84, 32)    2624      
_________________________________________________________________
batch_normalization_11 (Batc (None, 18, 84, 84, 32)    128       
_________________________________________________________________
activation_11 (Activation)   (None, 18, 84, 84, 32)    0         
_________________________________________________________________
conv3d_12 (Conv3D)           (None, 18, 84, 84, 64)    55360     
_________________________________________________________________
batch_normalization_12 (Batc (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation_12 (Activation)   (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 9, 42, 84, 64)    

In [23]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [24]:
model_4.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 55s 3s/step - loss: 5.2266 - categorical_accuracy: 0.1900 - val_loss: 1.6434 - val_categorical_accuracy: 0.2300

Epoch 00001: val_loss improved from inf to 1.64344, saving model to model_init_2021-10-2714_06_11.512848\model-00001-5.22657-0.19005-1.64344-0.23000.h5
Epoch 2/50
21/21 [==============================] - 49s 2s/step - loss: 4.9184 - categorical_accuracy: 0.2278 - val_loss: 1.8477 - val_categorical_accuracy: 0.2200

Epoch 00002: val_loss did not improve from 1.64344
Epoch 3/50
21/21 [==============================] - 52s 3s/step - loss: 4.4334 - categorical_accuracy: 0.2112 - val_loss: 2.2449 - val_categorical_accuracy: 0.2100

Epoch 00003: val_loss did not improve from 1.64344
Epoch 4/50
21/21 [==============================] - 51s 3s/step - loss: 3.9639 - categorical_accuracy: 0.2474 - val_loss: 2.6584 - val_categorical_accuracy: 0.2100

Epoch 00004: val_loss did not improve from 1.64344
Epoch 5/50
21/21 [=================

In [ ]:
# Although, the difference between categorical accuracy and validation accuracy is not much, overall performance of
# model 4 is not good

In [ ]:
# Running model 4 with 'relu' as activation function:

In [25]:
#write your model here
model_5 = Sequential()

model_5.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_5.add(BatchNormalization())
model_5.add(Activation('relu'))
model_5.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_5.add(BatchNormalization())
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(2,2,1)))
model_5.add(Dropout(0.25))

model_5.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_5.add(BatchNormalization())
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(2,2,2)))
model_5.add(Dropout(0.25))

model_5.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_5.add(BatchNormalization())
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(2,2,1)))
model_5.add(Dropout(0.25))

model_5.add(Flatten())
model_5.add(Dropout(0.5))
model_5.add(Dense(1024, activation='relu'))
model_5.add(Dropout(0.5))
model_5.add(Dense(512, activation='relu'))
model_5.add(Dropout(0.5))
model_5.add(Dense(5, activation='softmax'))

In [26]:
model_5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_5.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_15 (Conv3D)           (None, 18, 84, 84, 32)    2624      
_________________________________________________________________
batch_normalization_15 (Batc (None, 18, 84, 84, 32)    128       
_________________________________________________________________
activation_15 (Activation)   (None, 18, 84, 84, 32)    0         
_________________________________________________________________
conv3d_16 (Conv3D)           (None, 18, 84, 84, 64)    55360     
_________________________________________________________________
batch_normalization_16 (Batc (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation_16 (Activation)   (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 9, 42, 84, 64)    

In [27]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [28]:
model_5.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 52s 3s/step - loss: 5.0641 - categorical_accuracy: 0.1946 - val_loss: 1.6071 - val_categorical_accuracy: 0.2600

Epoch 00001: val_loss improved from inf to 1.60713, saving model to model_init_2021-10-2714_06_11.512848\model-00001-5.06414-0.19457-1.60713-0.26000.h5
Epoch 2/50
21/21 [==============================] - 52s 3s/step - loss: 4.3366 - categorical_accuracy: 0.1946 - val_loss: 1.6149 - val_categorical_accuracy: 0.2000

Epoch 00002: val_loss did not improve from 1.60713
Epoch 3/50
21/21 [==============================] - 51s 3s/step - loss: 4.1028 - categorical_accuracy: 0.1991 - val_loss: 1.6387 - val_categorical_accuracy: 0.1900

Epoch 00003: val_loss did not improve from 1.60713
Epoch 4/50
21/21 [==============================] - 50s 2s/step - loss: 3.6478 - categorical_accuracy: 0.2066 - val_loss: 1.6696 - val_categorical_accuracy: 0.1900

Epoch 00004: val_loss did not improve from 1.60713
Epoch 5/50
21/21 [=================

In [ ]:
# We can see that the performance of 'elu' activation on the same model is much better than the 'relu' activation --> on the
# given data (After 50 epochs).

In [ ]:
# Adding 25% dropout after the 3rd MaxPool layer:

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
model_6 = Sequential()

model_6.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_6.add(BatchNormalization())
model_6.add(Activation('elu'))
model_6.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model_6.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_6.add(BatchNormalization())
model_6.add(Activation('elu'))
model_6.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model_6.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_6.add(BatchNormalization())
model_6.add(Activation('elu'))
model_6.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))
model_6.add(Dropout(0.25))

model_6.add(Flatten())
model_6.add(Dropout(0.5))
model_6.add(Dense(512, activation='elu'))
model_6.add(Dropout(0.5))
model_6.add(Dense(5, activation='softmax'))

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

optimiser = keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer

model_6.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_6.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation (Activation)      (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 42, 84, 64)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 9, 42, 84, 128)    221312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 42, 84, 128)    512       
_________________________________________________________________
activation_1 (Activation)    (None, 9, 42, 84, 128)    0

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [11]:
model_6.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:\Users\Amit\Documents\Academics\upGrad\Deep Learning\Project_data\train ; batch size = 32
Epoch 1/50
21/21 [==============================] - 65s 3s/step - loss: 9.4492 - categorical_accuracy: 0.2821 - val_loss: 2.0736 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss improved from inf to 2.07357, saving model to model_init_2021-10-2718_08_28.064000\model-00001-9.44915-0.28205-2.07357-0.21000.h5
Epoch 2/50
21/21 [==============================] - 51s 3s/step - loss: 1.7034 - categorical_accuracy: 0.4087 - val_loss: 2.2690 - val_categorical_accuracy: 0.2200

Epoch 00002: val_loss did not improve from 2.07357
Epoch 3/50
21/21 [==============================] - 51s 3s/step - loss: 1.3762 - categorical_accuracy: 0.5143 - val_loss: 3.6814 - val_categorical_accuracy: 0.2200

Epoch 00003: val_loss did not improve from 2.07357

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/50
21/21 [==============================] - 49s 2s/step

RuntimeError: Can't decrement id ref count (unable to extend file properly)

In [ ]:
# We got runtime error after the 30th epoch of model 6 as the .h5 files reached 109 GB and filled up the hard disk
# We would not be re-running model 6 again as it was not giving good results (difference between the train and val
# accuracies was consistently high from epoch 1 to epoch 30)

In [ ]:
# Using model 6 and adding 25% dropouts after the 2nd MaxPool layer:

In [12]:
#write your model here
model_7 = Sequential()

model_7.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_7.add(BatchNormalization())
model_7.add(Activation('elu'))
model_7.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))


model_7.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_7.add(BatchNormalization())
model_7.add(Activation('elu'))
model_7.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_7.add(Dropout(0.25))


model_7.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_7.add(BatchNormalization())
model_7.add(Activation('elu'))
model_7.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))
model_7.add(Dropout(0.25))

model_7.add(Flatten())
model_7.add(Dropout(0.5))
model_7.add(Dense(512, activation='elu'))
model_7.add(Dropout(0.5))
model_7.add(Dense(5, activation='softmax'))

In [13]:
model_7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_7.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 18, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization_3 (Batch (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation_3 (Activation)    (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 9, 42, 84, 64)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 9, 42, 84, 128)    221312    
_________________________________________________________________
batch_normalization_4 (Batch (None, 9, 42, 84, 128)    512       
_________________________________________________________________
activation_4 (Activation)    (None, 9, 42, 84, 128)   

In [14]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [16]:
model_7.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 49s 2s/step - loss: 3.6757 - categorical_accuracy: 0.2443 - val_loss: 1.7234 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss did not improve from 1.62567
Epoch 2/50
21/21 [==============================] - 50s 2s/step - loss: 3.1555 - categorical_accuracy: 0.2881 - val_loss: 1.9417 - val_categorical_accuracy: 0.2100

Epoch 00002: val_loss did not improve from 1.62567
Epoch 3/50
21/21 [==============================] - 52s 3s/step - loss: 2.7861 - categorical_accuracy: 0.3228 - val_loss: 2.1360 - val_categorical_accuracy: 0.2200

Epoch 00003: val_loss did not improve from 1.62567
Epoch 4/50
21/21 [==============================] - 52s 3s/step - loss: 2.5230 - categorical_accuracy: 0.3756 - val_loss: 2.8042 - val_categorical_accuracy: 0.1700

Epoch 00004: val_loss did not improve from 1.62567
Epoch 5/50
21/21 [==============================] - 50s 2s/step - loss: 2.5025 - categorical_accuracy: 0.3484 - val_loss: 3.0570 - val_c

In [ ]:
# Model 7 is slightly underfitting

In [17]:
#write your model here
model_8 = Sequential()

model_8.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
# model_8.add(BatchNormalization())
model_8.add(Activation('relu'))
model_8.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
# model_8.add(BatchNormalization())
model_8.add(Activation('relu'))
model_8.add(MaxPooling3D(pool_size=(2,2,1)))
model_8.add(Dropout(0.25))

model_8.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
# model_8.add(BatchNormalization())
model_8.add(Activation('relu'))
model_8.add(MaxPooling3D(pool_size=(2,2,2)))
model_8.add(Dropout(0.25))

model_8.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
# model_8.add(BatchNormalization())
model_8.add(Activation('relu'))
model_8.add(MaxPooling3D(pool_size=(2,2,1)))
model_8.add(Dropout(0.25))

model_8.add(Flatten())
model_8.add(Dropout(0.5))
model_8.add(Dense(1024, activation='relu'))
model_8.add(Dropout(0.5))
model_8.add(Dense(512, activation='relu'))
model_8.add(Dropout(0.5))
model_8.add(Dense(5, activation='softmax'))

In [18]:
model_8.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_8.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_6 (Conv3D)            (None, 18, 84, 84, 32)    2624      
_________________________________________________________________
activation_6 (Activation)    (None, 18, 84, 84, 32)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 18, 84, 84, 64)    55360     
_________________________________________________________________
activation_7 (Activation)    (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 9, 42, 84, 64)     0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 9, 42, 84, 64)     0         
_________________________________________________________________
conv3d_8 (Conv3D)            (None, 9, 42, 84, 128)   

In [19]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [20]:
model_8.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 56s 3s/step - loss: 1.6353 - categorical_accuracy: 0.2142 - val_loss: 1.6145 - val_categorical_accuracy: 0.1200

Epoch 00001: val_loss improved from inf to 1.61447, saving model to model_init_2021-10-2718_08_28.064000\model-00001-1.63531-0.21418-1.61447-0.12000.h5
Epoch 2/50
21/21 [==============================] - 51s 3s/step - loss: 1.6436 - categorical_accuracy: 0.1991 - val_loss: 1.6083 - val_categorical_accuracy: 0.1500

Epoch 00002: val_loss improved from 1.61447 to 1.60834, saving model to model_init_2021-10-2718_08_28.064000\model-00002-1.64358-0.19910-1.60834-0.15000.h5
Epoch 3/50
21/21 [==============================] - 49s 2s/step - loss: 1.6569 - categorical_accuracy: 0.1840 - val_loss: 1.6112 - val_categorical_accuracy: 0.1800

Epoch 00003: val_loss did not improve from 1.60834
Epoch 4/50
21/21 [==============================] - 51s 3s/step - loss: 1.6490 - categorical_accuracy: 0.1644 - val_loss: 1.6106 - val_categorical

In [25]:
# Comparing the results of models 5 and 8 after 50 epochs:

# Model 5:
# loss: 1.6184 - categorical_accuracy: 0.3695 - val_loss: 1.5264 - val_categorical_accuracy: 0.2300

# Model 8:
#  loss: 1.6278 - categorical_accuracy: 0.1931 - val_loss: 1.6015 - val_categorical_accuracy: 0.2300

# We can see that removing batch normalization has adversely affected the categorical accuracy and validation loss.
# Moreover, both of these models (model 5 and model 8) are pretty much useless for the data that we have.

In [21]:
#write your model here
model_9 = Sequential()

model_9.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_9.add(BatchNormalization())
model_9.add(Activation('relu'))
model_9.add(MaxPooling3D(pool_size=(2,2,2)))
# model_9.add(Dropout(0.25))

model_9.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_9.add(BatchNormalization())
model_9.add(Activation('relu'))
model_9.add(MaxPooling3D(pool_size=(2,2,2)))
# model_9.add(Dropout(0.25))

model_9.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_9.add(BatchNormalization())
model_9.add(Activation('relu'))
model_9.add(MaxPooling3D(pool_size=(2,2,2)))
# model_9.add(Dropout(0.25))

model_9.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_9.add(BatchNormalization())
model_9.add(Activation('relu'))
model_9.add(MaxPooling3D(pool_size=(2,2,2)))
# model_9.add(Dropout(0.25))

model_9.add(Flatten())
model_9.add(Dropout(0.5))
model_9.add(Dense(1024, activation='relu'))
model_9.add(Dropout(0.5))
model_9.add(Dense(512, activation='relu'))
model_9.add(Dropout(0.5))
model_9.add(Dense(5, activation='softmax'))

In [22]:
model_9.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_9.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 18, 84, 84, 32)    2624      
_________________________________________________________________
batch_normalization_6 (Batch (None, 18, 84, 84, 32)    128       
_________________________________________________________________
activation_10 (Activation)   (None, 18, 84, 84, 32)    0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 9, 42, 42, 32)     0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 9, 42, 42, 64)     55360     
_________________________________________________________________
batch_normalization_7 (Batch (None, 9, 42, 42, 64)     256       
_________________________________________________________________
activation_11 (Activation)   (None, 9, 42, 42, 64)    

In [23]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [24]:
model_9.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 49s 2s/step - loss: 4.9019 - categorical_accuracy: 0.2262 - val_loss: 1.6140 - val_categorical_accuracy: 0.2300

Epoch 00001: val_loss improved from inf to 1.61395, saving model to model_init_2021-10-2718_08_28.064000\model-00001-4.90189-0.22624-1.61395-0.23000.h5
Epoch 2/50
21/21 [==============================] - 51s 3s/step - loss: 4.8916 - categorical_accuracy: 0.1825 - val_loss: 1.6170 - val_categorical_accuracy: 0.2200

Epoch 00002: val_loss did not improve from 1.61395
Epoch 3/50
21/21 [==============================] - 51s 3s/step - loss: 4.9116 - categorical_accuracy: 0.1765 - val_loss: 1.6295 - val_categorical_accuracy: 0.2200

Epoch 00003: val_loss did not improve from 1.61395
Epoch 4/50
21/21 [==============================] - 50s 2s/step - loss: 4.3051 - categorical_accuracy: 0.2142 - val_loss: 1.6235 - val_categorical_accuracy: 0.2100

Epoch 00004: val_loss did not improve from 1.61395
Epoch 5/50
21/21 [=================

In [ ]:
# Model 9 is underfitting

In [26]:
#write your model here
model_10 = Sequential()

model_10.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
# model_10.add(BatchNormalization())
model_10.add(Activation('relu'))
model_10.add(MaxPooling3D(pool_size=(2,2,2)))
model_10.add(Dropout(0.25))

model_10.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
# model_10.add(BatchNormalization())
model_10.add(Activation('relu'))
model_10.add(MaxPooling3D(pool_size=(2,2,2)))
model_10.add(Dropout(0.25))

model_10.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
# model_10.add(BatchNormalization())
model_10.add(Activation('relu'))
model_10.add(MaxPooling3D(pool_size=(2,2,2)))
model_10.add(Dropout(0.25))

model_10.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
# model_10.add(BatchNormalization())
model_10.add(Activation('relu'))
model_10.add(MaxPooling3D(pool_size=(2,2,2)))
model_10.add(Dropout(0.25))

model_10.add(Flatten())
model_10.add(Dropout(0.5))
model_10.add(Dense(1024, activation='relu'))
model_10.add(Dropout(0.5))
model_10.add(Dense(512, activation='relu'))
model_10.add(Dropout(0.5))
model_10.add(Dense(5, activation='softmax'))

In [27]:
model_10.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_10.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_14 (Conv3D)           (None, 18, 84, 84, 32)    2624      
_________________________________________________________________
activation_14 (Activation)   (None, 18, 84, 84, 32)    0         
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 9, 42, 42, 32)     0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 9, 42, 42, 32)     0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 9, 42, 42, 64)     55360     
_________________________________________________________________
activation_15 (Activation)   (None, 9, 42, 42, 64)     0         
_________________________________________________________________
max_pooling3d_14 (MaxPooling (None, 4, 21, 21, 64)    

In [28]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [29]:
model_10.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 51s 3s/step - loss: 1.8074 - categorical_accuracy: 0.1765 - val_loss: 1.6108 - val_categorical_accuracy: 0.2600

Epoch 00001: val_loss improved from inf to 1.61079, saving model to model_init_2021-10-2718_08_28.064000\model-00001-1.80741-0.17647-1.61079-0.26000.h5
Epoch 2/50
21/21 [==============================] - 50s 2s/step - loss: 1.7774 - categorical_accuracy: 0.2066 - val_loss: 1.6141 - val_categorical_accuracy: 0.2300

Epoch 00002: val_loss did not improve from 1.61079
Epoch 3/50
21/21 [==============================] - 48s 2s/step - loss: 1.7637 - categorical_accuracy: 0.2066 - val_loss: 1.6107 - val_categorical_accuracy: 0.2600

Epoch 00003: val_loss improved from 1.61079 to 1.61074, saving model to model_init_2021-10-2718_08_28.064000\model-00003-1.76366-0.20664-1.61074-0.26000.h5
Epoch 4/50
21/21 [==============================] - 51s 3s/step - loss: 1.7330 - categorical_accuracy: 0.2006 - val_loss: 1.6115 - val_categorical

In [ ]:
# Model 10 also appears to be a bad model for the given data

In [30]:
#write your model here
model_11 = Sequential()

model_11.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_11.add(BatchNormalization())
model_11.add(Activation('relu'))
model_11.add(MaxPooling3D(pool_size=(2,2,2)))
model_11.add(Dropout(0.25))

model_11.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_11.add(BatchNormalization())
model_11.add(Activation('relu'))
model_11.add(MaxPooling3D(pool_size=(2,2,2)))
model_11.add(Dropout(0.25))

model_11.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_11.add(BatchNormalization())
model_11.add(Activation('relu'))
model_11.add(MaxPooling3D(pool_size=(2,2,2)))
model_11.add(Dropout(0.25))

model_11.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_11.add(BatchNormalization())
model_11.add(Activation('relu'))
model_11.add(MaxPooling3D(pool_size=(2,2,2)))
model_11.add(Dropout(0.25))

model_11.add(Flatten())
model_11.add(Dropout(0.5))
model_11.add(Dense(1024, activation='relu'))
model_11.add(Dropout(0.5))
model_11.add(Dense(512, activation='relu'))
model_11.add(Dropout(0.5))
model_11.add(Dense(5, activation='softmax'))

In [31]:
model_11.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_11.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_18 (Conv3D)           (None, 18, 84, 84, 32)    2624      
_________________________________________________________________
batch_normalization_10 (Batc (None, 18, 84, 84, 32)    128       
_________________________________________________________________
activation_18 (Activation)   (None, 18, 84, 84, 32)    0         
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 9, 42, 42, 32)     0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 9, 42, 42, 32)     0         
_________________________________________________________________
conv3d_19 (Conv3D)           (None, 9, 42, 42, 64)     55360     
_________________________________________________________________
batch_normalization_11 (Batc (None, 9, 42, 42, 64)    

In [32]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [33]:
model_11.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 50s 2s/step - loss: 5.6400 - categorical_accuracy: 0.1900 - val_loss: 1.6126 - val_categorical_accuracy: 0.1600

Epoch 00001: val_loss improved from inf to 1.61260, saving model to model_init_2021-10-2718_08_28.064000\model-00001-5.64003-0.19005-1.61260-0.16000.h5
Epoch 2/50
21/21 [==============================] - 51s 3s/step - loss: 5.1649 - categorical_accuracy: 0.2006 - val_loss: 1.6143 - val_categorical_accuracy: 0.1600

Epoch 00002: val_loss did not improve from 1.61260
Epoch 3/50
21/21 [==============================] - 50s 3s/step - loss: 5.5630 - categorical_accuracy: 0.1780 - val_loss: 1.6178 - val_categorical_accuracy: 0.1600

Epoch 00003: val_loss did not improve from 1.61260
Epoch 4/50
21/21 [==============================] - 52s 3s/step - loss: 5.9188 - categorical_accuracy: 0.1735 - val_loss: 1.6259 - val_categorical_accuracy: 0.1500

Epoch 00004: val_loss did not improve from 1.61260
Epoch 5/50
21/21 [=================

In [ ]:
# Performance of model 11 is too poor

In [ ]:
# Out of all the models (1 to 11) we have seen till now, model 1 appeared to perform the best. We would be tweaking model_1, a
# little and trying to run that model again:

In [34]:
#write your model here
model_12 = Sequential()

model_12.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_12.add(BatchNormalization())
model_12.add(Activation('elu'))
model_12.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))


model_12.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_12.add(BatchNormalization())
model_12.add(Activation('elu'))
model_12.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


model_12.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model_12.add(BatchNormalization())
model_12.add(Activation('elu'))
model_12.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))


model_12.add(Flatten())
model_12.add(Dropout(0.5))
model_12.add(Dense(1024, activation='elu'))
model_12.add(Dropout(0.5))
model_12.add(Dense(512, activation='elu'))
model_12.add(Dropout(0.5))
model_12.add(Dense(5, activation='softmax'))

In [35]:
model_12.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_12.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_22 (Conv3D)           (None, 18, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization_14 (Batc (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation_22 (Activation)   (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 9, 42, 84, 64)     0         
_________________________________________________________________
conv3d_23 (Conv3D)           (None, 9, 42, 84, 128)    221312    
_________________________________________________________________
batch_normalization_15 (Batc (None, 9, 42, 84, 128)    512       
_________________________________________________________________
activation_23 (Activation)   (None, 9, 42, 84, 128)   

In [36]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [37]:
model_12.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 53s 3s/step - loss: 4.4650 - categorical_accuracy: 0.2081 - val_loss: 1.5935 - val_categorical_accuracy: 0.2800

Epoch 00001: val_loss improved from inf to 1.59351, saving model to model_init_2021-10-2718_08_28.064000\model-00001-4.46497-0.20814-1.59351-0.28000.h5
Epoch 2/50
21/21 [==============================] - 52s 3s/step - loss: 3.6054 - categorical_accuracy: 0.2745 - val_loss: 1.5833 - val_categorical_accuracy: 0.2300

Epoch 00002: val_loss improved from 1.59351 to 1.58334, saving model to model_init_2021-10-2718_08_28.064000\model-00002-3.60539-0.27451-1.58334-0.23000.h5
Epoch 3/50
21/21 [==============================] - 48s 2s/step - loss: 3.4056 - categorical_accuracy: 0.2745 - val_loss: 1.6386 - val_categorical_accuracy: 0.2000

Epoch 00003: val_loss did not improve from 1.58334
Epoch 4/50
21/21 [==============================] - 51s 3s/step - loss: 2.9027 - categorical_accuracy: 0.3318 - val_loss: 1.8999 - val_categorical

Epoch 34/50
21/21 [==============================] - 50s 3s/step - loss: 1.2144 - categorical_accuracy: 0.6139 - val_loss: 0.8550 - val_categorical_accuracy: 0.6700

Epoch 00034: val_loss improved from 0.91220 to 0.85504, saving model to model_init_2021-10-2718_08_28.064000\model-00034-1.21436-0.61388-0.85504-0.67000.h5
Epoch 35/50
21/21 [==============================] - 49s 2s/step - loss: 1.2707 - categorical_accuracy: 0.5867 - val_loss: 0.8530 - val_categorical_accuracy: 0.7000

Epoch 00035: val_loss improved from 0.85504 to 0.85296, saving model to model_init_2021-10-2718_08_28.064000\model-00035-1.27074-0.58673-0.85296-0.70000.h5
Epoch 36/50
21/21 [==============================] - 51s 3s/step - loss: 1.1870 - categorical_accuracy: 0.6109 - val_loss: 0.8550 - val_categorical_accuracy: 0.7000

Epoch 00036: val_loss did not improve from 0.85296
Epoch 37/50
21/21 [==============================] - 51s 3s/step - loss: 1.1631 - categorical_accuracy: 0.6033 - val_loss: 0.8673 - val_cat

In [ ]:
# Clearly, models 1 stands out. Thus, we would be re-running models 1 and 12 as conv3d_model_1 with the parameter
# save_best = False

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
conv3d_model_1 = Sequential()

conv3d_model_1.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
conv3d_model_1.add(BatchNormalization())
conv3d_model_1.add(Activation('elu'))
conv3d_model_1.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))


conv3d_model_1.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
conv3d_model_1.add(BatchNormalization())
conv3d_model_1.add(Activation('elu'))
conv3d_model_1.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


conv3d_model_1.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
conv3d_model_1.add(BatchNormalization())
conv3d_model_1.add(Activation('elu'))
conv3d_model_1.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))


conv3d_model_1.add(Flatten())
conv3d_model_1.add(Dropout(0.5))
conv3d_model_1.add(Dense(512, activation='elu'))
conv3d_model_1.add(Dropout(0.5))
conv3d_model_1.add(Dense(5, activation='softmax'))

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

optimiser = keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True) #write your optimizer

conv3d_model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(conv3d_model_1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation (Activation)      (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 9, 42, 84, 64)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 9, 42, 84, 128)    221312    
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 42, 84, 128)    512       
_________________________________________________________________
activation_1 (Activation)    (None, 9, 42, 84, 128)    0

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [11]:
conv3d_model_1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:\Users\Amit\Documents\Academics\upGrad\Deep Learning\Project_data\train ; batch size = 32
Epoch 1/50
21/21 [==============================] - 63s 3s/step - loss: 7.8384 - categorical_accuracy: 0.2926 - val_loss: 3.0341 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model_init_2021-10-2723_54_21.814102\model-00001-7.83838-0.29261-3.03408-0.18000.h5
Epoch 2/50
21/21 [==============================] - 52s 3s/step - loss: 1.7314 - categorical_accuracy: 0.4042 - val_loss: 2.4487 - val_categorical_accuracy: 0.1800

Epoch 00002: saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.73141-0.40422-2.44870-0.18000.h5
Epoch 3/50
21/21 [==============================] - 52s 3s/step - loss: 1.5232 - categorical_accuracy: 0.4842 - val_loss: 4.4316 - val_categorical_accuracy: 0.1400

Epoch 00003: saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.52325-0.48416-4.43162-0.14000.h5
Epoch 4/50
21/21 [==============================] - 50s 2s/

In [ ]:
# Epoch 35: loss: 0.4459 - categorical_accuracy: 0.8145 - val_loss: 0.9167 - val_categorical_accuracy: 0.7900

In [12]:
#write your model here
conv3d_model_2 = Sequential()

conv3d_model_2.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
conv3d_model_2.add(BatchNormalization())
conv3d_model_2.add(Activation('elu'))
conv3d_model_2.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))


conv3d_model_2.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
conv3d_model_2.add(BatchNormalization())
conv3d_model_2.add(Activation('elu'))
conv3d_model_2.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))


conv3d_model_2.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
conv3d_model_2.add(BatchNormalization())
conv3d_model_2.add(Activation('elu'))
conv3d_model_2.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))


conv3d_model_2.add(Flatten())
conv3d_model_2.add(Dropout(0.5))
conv3d_model_2.add(Dense(1024, activation='elu'))
conv3d_model_2.add(Dropout(0.5))
conv3d_model_2.add(Dense(512, activation='elu'))
conv3d_model_2.add(Dropout(0.5))
conv3d_model_2.add(Dense(5, activation='softmax'))

In [13]:
conv3d_model_2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(conv3d_model_2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 18, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization_3 (Batch (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation_3 (Activation)    (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 9, 42, 84, 64)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 9, 42, 84, 128)    221312    
_________________________________________________________________
batch_normalization_4 (Batch (None, 9, 42, 84, 128)    512       
_________________________________________________________________
activation_4 (Activation)    (None, 9, 42, 84, 128)   

In [14]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [15]:
conv3d_model_2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 51s 3s/step - loss: 4.3874 - categorical_accuracy: 0.1825 - val_loss: 1.6406 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2021-10-2723_54_21.814102\model-00001-4.38737-0.18250-1.64055-0.20000.h5
Epoch 2/50
21/21 [==============================] - 49s 2s/step - loss: 3.6685 - categorical_accuracy: 0.2308 - val_loss: 1.6548 - val_categorical_accuracy: 0.2900

Epoch 00002: saving model to model_init_2021-10-2723_54_21.814102\model-00002-3.66855-0.23077-1.65484-0.29000.h5
Epoch 3/50
21/21 [==============================] - 52s 3s/step - loss: 3.5081 - categorical_accuracy: 0.2534 - val_loss: 1.7666 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2021-10-2723_54_21.814102\model-00003-3.50814-0.25339-1.76657-0.21000.h5
Epoch 4/50
21/21 [==============================] - 51s 3s/step - loss: 2.9204 - categorical_accuracy: 0.3107 - val_loss: 2.0143 - val_categorical_accuracy: 0.2100

E


Epoch 00030: saving model to model_init_2021-10-2723_54_21.814102\model-00030-1.23438-0.60181-0.98766-0.58000.h5
Epoch 31/50
21/21 [==============================] - 52s 3s/step - loss: 1.1964 - categorical_accuracy: 0.5958 - val_loss: 1.0329 - val_categorical_accuracy: 0.6100

Epoch 00031: saving model to model_init_2021-10-2723_54_21.814102\model-00031-1.19642-0.59578-1.03293-0.61000.h5
Epoch 32/50
21/21 [==============================] - 53s 3s/step - loss: 1.3109 - categorical_accuracy: 0.5928 - val_loss: 0.8835 - val_categorical_accuracy: 0.6900

Epoch 00032: saving model to model_init_2021-10-2723_54_21.814102\model-00032-1.31094-0.59276-0.88346-0.69000.h5
Epoch 33/50
21/21 [==============================] - 53s 3s/step - loss: 1.3628 - categorical_accuracy: 0.5596 - val_loss: 0.8261 - val_categorical_accuracy: 0.6700

Epoch 00033: saving model to model_init_2021-10-2723_54_21.814102\model-00033-1.36282-0.55958-0.82606-0.67000.h5
Epoch 34/50
21/21 [==============================

In [ ]:
# After observing the results of the two models, we conclude that the result obtained in the 35th epoch of model conv3d_model_1
# is the best result so far. Thus, we would be uploading its .h5 file for evaluation.

# CNN - RNN Models (Without Transfer Learning)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [16]:
# source_path = train_path i.e. path of 663 folders
# folder_list = train_doc i.e. csv file
# batch_size = 64

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    # It is not possible to work with all the 30 images, as it will take too long processing time.
    # So lets choose randomly 20 images, as this is more computationally expensive
    img_idx = [1,2,4,6,10,11,12,14,15,16,17,18,19,20,22,23,24,27,28,29] #create a list of image numbers you want to use for a particular video(incase if u want to try with lesser images)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,20,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image[:,20:140,:],(84,84)).astype(np.float32)
                    normalizedImg = image/255.0                    
                    
                    batch_data[folder,idx,:,:,0] = (normalizedImg[:,:,0]) #normalise and feed in the image # divide by 255.0
                    batch_data[folder,idx,:,:,1] = (normalizedImg[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (normalizedImg[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size)!=0:
            batch_data = np.zeros((len(t)%batch_size,20,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image[:,20:140,:],(84,84)).astype(np.float32)
                    normalizedImg = image/255.0                    
                    
                    batch_data[folder,idx,:,:,0] = (normalizedImg[:,:,0]) #normalise and feed in the image # divide by 255.0
                    batch_data[folder,idx,:,:,1] = (normalizedImg[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (normalizedImg[:,:,2]) #normalise and feed in the imagee
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels

In [17]:
num_epochs = 50

In [18]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here

model_13 = Sequential()

model_13.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))


model_13.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_13.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_13.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_13.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_13.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_13.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_13.add(TimeDistributed(BatchNormalization()))
model_13.add(Dropout(0.25))

model_13.add(TimeDistributed(Flatten()))

model_13.add(Dense(128, activation='relu'))
model_13.add(Dropout(0.25))
model_13.add(Dense(64, activation='relu'))
model_13.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model_13.add(GRU(128, return_sequences=False))
model_13.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [19]:
from tensorflow.keras.optimizers import Adam

optimiser_2 = Adam(0.001) #write your optimizer

model_13.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_13.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 5, 5, 64)     

In [20]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [21]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [22]:
model_13.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:\Users\Amit\Documents\Academics\upGrad\Deep Learning\Project_data\train ; batch size = 32
Epoch 1/50
21/21 [==============================] - 59s 3s/step - loss: 1.4560 - categorical_accuracy: 0.3590 - val_loss: 1.7117 - val_categorical_accuracy: 0.2300

Epoch 00001: val_loss improved from inf to 1.71168, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.45598-0.35897-1.71168-0.23000.h5
Epoch 2/50
21/21 [==============================] - 57s 3s/step - loss: 1.2007 - categorical_accuracy: 0.4827 - val_loss: 1.6111 - val_categorical_accuracy: 0.2800

Epoch 00002: val_loss improved from 1.71168 to 1.61108, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.20070-0.48265-1.61108-0.28000.h5
Epoch 3/50
21/21 [==============================] - 57s 3s/step - loss: 1.0113 - categorical_accuracy: 0.5807 - val_loss: 1.5984 - val_categorical_accuracy: 0.2100

Epoch 00003: val_loss improved from 1.61108 to 1.59839, saving model to model_init_2021-10

21/21 [==============================] - 56s 3s/step - loss: 0.0059 - categorical_accuracy: 1.0000 - val_loss: 1.2474 - val_categorical_accuracy: 0.7000

Epoch 00031: val_loss did not improve from 0.86238
Epoch 32/50
21/21 [==============================] - 54s 3s/step - loss: 0.0059 - categorical_accuracy: 1.0000 - val_loss: 1.3343 - val_categorical_accuracy: 0.6300

Epoch 00032: val_loss did not improve from 0.86238
Epoch 33/50
21/21 [==============================] - 56s 3s/step - loss: 0.0081 - categorical_accuracy: 1.0000 - val_loss: 1.3839 - val_categorical_accuracy: 0.6400

Epoch 00033: val_loss did not improve from 0.86238
Epoch 34/50
21/21 [==============================] - 56s 3s/step - loss: 0.0079 - categorical_accuracy: 1.0000 - val_loss: 1.3323 - val_categorical_accuracy: 0.6400

Epoch 00034: val_loss did not improve from 0.86238
Epoch 35/50
21/21 [==============================] - 56s 3s/step - loss: 0.0063 - categorical_accuracy: 1.0000 - val_loss: 1.5141 - val_categori

In [ ]:
# We can see that this model is massively overfitting

In [ ]:
# Trying with 16, 32, 64 and 128 in Conv2D. Also, increasing the dropout percentage from 25% to 50% in dense layers:

In [23]:
#write your model here

model_14 = Sequential()

model_14.add(TimeDistributed(Conv2D(16, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))


model_14.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_14.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_14.add(TimeDistributed(Conv2D(64, (3,3),padding='same', activation='relu')))
model_14.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_14.add(TimeDistributed(Conv2D(128, (2,2),padding='same', activation='relu')))
model_14.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_14.add(TimeDistributed(BatchNormalization()))
model_14.add(Dropout(0.25))

model_14.add(TimeDistributed(Flatten()))

model_14.add(Dense(128, activation='relu'))
model_14.add(Dropout(0.5))
model_14.add(Dense(64, activation='relu'))
model_14.add(Dropout(0.5))

## using GRU as the RNN model along with softmax as our last layer.
model_14.add(GRU(128, return_sequences=False))
model_14.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [24]:
model_14.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_14.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_9 (TimeDist (None, 20, 42, 42, 16)    448       
_________________________________________________________________
time_distributed_10 (TimeDis (None, 20, 42, 42, 32)    4640      
_________________________________________________________________
time_distributed_11 (TimeDis (None, 20, 21, 21, 32)    0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 20, 21, 21, 64)    18496     
_________________________________________________________________
time_distributed_13 (TimeDis (None, 20, 10, 10, 64)    0         
_________________________________________________________________
time_distributed_14 (TimeDis (None, 20, 10, 10, 128)   32896     
_________________________________________________________________
time_distributed_15 (TimeDis (None, 20, 5, 5, 128)    

In [25]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [26]:
model_14.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 58s 3s/step - loss: 1.6020 - categorical_accuracy: 0.2413 - val_loss: 1.6074 - val_categorical_accuracy: 0.1800

Epoch 00001: val_loss improved from inf to 1.60743, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.60198-0.24133-1.60743-0.18000.h5
Epoch 2/50
21/21 [==============================] - 54s 3s/step - loss: 1.5496 - categorical_accuracy: 0.3047 - val_loss: 1.6034 - val_categorical_accuracy: 0.3000

Epoch 00002: val_loss improved from 1.60743 to 1.60343, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.54962-0.30468-1.60343-0.30000.h5
Epoch 3/50
21/21 [==============================] - 56s 3s/step - loss: 1.5190 - categorical_accuracy: 0.3258 - val_loss: 1.6007 - val_categorical_accuracy: 0.2800

Epoch 00003: val_loss improved from 1.60343 to 1.60067, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.51904-0.32579-1.60067-0.28000.h5
Epoch 4/50
21/21 [=======================

21/21 [==============================] - 56s 3s/step - loss: 1.0616 - categorical_accuracy: 0.6078 - val_loss: 1.2244 - val_categorical_accuracy: 0.5300

Epoch 00027: val_loss did not improve from 1.21677
Epoch 28/50
21/21 [==============================] - 56s 3s/step - loss: 1.0643 - categorical_accuracy: 0.5958 - val_loss: 1.1777 - val_categorical_accuracy: 0.5400

Epoch 00028: val_loss improved from 1.21677 to 1.17775, saving model to model_init_2021-10-2723_54_21.814102\model-00028-1.06430-0.59578-1.17775-0.54000.h5
Epoch 29/50
21/21 [==============================] - 56s 3s/step - loss: 1.0490 - categorical_accuracy: 0.6305 - val_loss: 1.2451 - val_categorical_accuracy: 0.4900

Epoch 00029: val_loss did not improve from 1.17775
Epoch 30/50
21/21 [==============================] - 54s 3s/step - loss: 1.0341 - categorical_accuracy: 0.6305 - val_loss: 1.1766 - val_categorical_accuracy: 0.5300

Epoch 00030: val_loss improved from 1.17775 to 1.17661, saving model to model_init_2021-10

In [ ]:
# Model_14 is overfitting

In [ ]:
# Adding one more conv2D layer to the first model and also using 50% dropouts in dense layers:

In [27]:
#write your model here

model_15 = Sequential()

model_15.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))
model_15.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_15.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_15.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_15.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_15.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_15.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_15.add(TimeDistributed(Conv2D(128, (2,2),padding='same', activation='relu')))
model_15.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_15.add(TimeDistributed(BatchNormalization()))
model_15.add(Dropout(0.25))

model_15.add(TimeDistributed(Flatten()))

model_15.add(Dense(128, activation='relu'))
model_15.add(Dropout(0.5))
model_15.add(Dense(64, activation='relu'))
model_15.add(Dropout(0.5))

## using GRU as the RNN model along with softmax as our last layer.
model_15.add(GRU(128, return_sequences=False))
model_15.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [28]:
model_15.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_15.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_18 (TimeDis (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_19 (TimeDis (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_20 (TimeDis (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_21 (TimeDis (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_22 (TimeDis (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_24 (TimeDis (None, 20, 5, 5, 64)     

In [29]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [30]:
model_15.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 57s 3s/step - loss: 1.5981 - categorical_accuracy: 0.2474 - val_loss: 1.6071 - val_categorical_accuracy: 0.2400

Epoch 00001: val_loss improved from inf to 1.60714, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.59811-0.24736-1.60714-0.24000.h5
Epoch 2/50
21/21 [==============================] - 55s 3s/step - loss: 1.5696 - categorical_accuracy: 0.2941 - val_loss: 1.6035 - val_categorical_accuracy: 0.2300

Epoch 00002: val_loss improved from 1.60714 to 1.60347, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.56963-0.29412-1.60347-0.23000.h5
Epoch 3/50
21/21 [==============================] - 56s 3s/step - loss: 1.5391 - categorical_accuracy: 0.3032 - val_loss: 1.6022 - val_categorical_accuracy: 0.1900

Epoch 00003: val_loss improved from 1.60347 to 1.60216, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.53910-0.30317-1.60216-0.19000.h5
Epoch 4/50
21/21 [=======================

In [ ]:
# Performance of Model_15 is not good enough

In [ ]:
# Trying model 1 with 50% dropouts

In [31]:
#write your model here

model_16 = Sequential()

model_16.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))


model_16.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_16.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_16.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_16.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_16.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_16.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_16.add(TimeDistributed(BatchNormalization()))
model_16.add(Dropout(0.25))

model_16.add(TimeDistributed(Flatten()))

model_16.add(Dense(128, activation='relu'))
model_16.add(Dropout(0.5))
model_16.add(Dense(64, activation='relu'))
model_16.add(Dropout(0.5))

## using GRU as the RNN model along with softmax as our last layer.
model_16.add(GRU(128, return_sequences=False))
model_16.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [33]:
model_16.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_16.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_29 (TimeDis (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_30 (TimeDis (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_31 (TimeDis (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_32 (TimeDis (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_33 (TimeDis (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_34 (TimeDis (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_35 (TimeDis (None, 20, 5, 5, 64)     

In [34]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [35]:
model_16.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 58s 3s/step - loss: 1.6289 - categorical_accuracy: 0.1946 - val_loss: 1.6070 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss improved from inf to 1.60701, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.62886-0.19457-1.60701-0.21000.h5
Epoch 2/50
21/21 [==============================] - 55s 3s/step - loss: 1.5866 - categorical_accuracy: 0.2624 - val_loss: 1.6055 - val_categorical_accuracy: 0.2200

Epoch 00002: val_loss improved from 1.60701 to 1.60550, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.58661-0.26244-1.60550-0.22000.h5
Epoch 3/50
21/21 [==============================] - 58s 3s/step - loss: 1.5501 - categorical_accuracy: 0.3167 - val_loss: 1.6023 - val_categorical_accuracy: 0.2300

Epoch 00003: val_loss improved from 1.60550 to 1.60233, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.55008-0.31674-1.60233-0.23000.h5
Epoch 4/50
21/21 [=======================

21/21 [==============================] - 56s 3s/step - loss: 1.1790 - categorical_accuracy: 0.5385 - val_loss: 1.2787 - val_categorical_accuracy: 0.4500

Epoch 00028: val_loss improved from 1.29689 to 1.27874, saving model to model_init_2021-10-2723_54_21.814102\model-00028-1.17900-0.53846-1.27874-0.45000.h5
Epoch 29/50
21/21 [==============================] - 56s 3s/step - loss: 1.1848 - categorical_accuracy: 0.5460 - val_loss: 1.2661 - val_categorical_accuracy: 0.5200

Epoch 00029: val_loss improved from 1.27874 to 1.26614, saving model to model_init_2021-10-2723_54_21.814102\model-00029-1.18476-0.54600-1.26614-0.52000.h5
Epoch 30/50
21/21 [==============================] - 54s 3s/step - loss: 1.1505 - categorical_accuracy: 0.5535 - val_loss: 1.2599 - val_categorical_accuracy: 0.5000

Epoch 00030: val_loss improved from 1.26614 to 1.25986, saving model to model_init_2021-10-2723_54_21.814102\model-00030-1.15048-0.55354-1.25986-0.50000.h5
Epoch 31/50
21/21 [===========================

In [ ]:
# Performance of model_16 is not good enough

In [ ]:
# Trying with batch normalization after every MaxPool Layer:

In [36]:
#write your model here

model_17 = Sequential()

model_17.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))
model_17.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_17.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_17.add(TimeDistributed(BatchNormalization()))

model_17.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_17.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_17.add(TimeDistributed(BatchNormalization()))

model_17.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_17.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_17.add(TimeDistributed(BatchNormalization()))
model_17.add(Dropout(0.25))

model_17.add(TimeDistributed(Flatten()))

model_17.add(Dense(128, activation='relu'))
model_17.add(Dropout(0.5))
model_17.add(Dense(64, activation='relu'))
model_17.add(Dropout(0.5))

## using GRU as the RNN model along with softmax as our last layer.
model_17.add(GRU(128, return_sequences=False))
model_17.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [37]:
model_17.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_17.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_38 (TimeDis (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_39 (TimeDis (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_40 (TimeDis (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_41 (TimeDis (None, 20, 21, 21, 16)    64        
_________________________________________________________________
time_distributed_42 (TimeDis (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_43 (TimeDis (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_44 (TimeDis (None, 20, 10, 10, 32)   

In [38]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [39]:
model_17.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 59s 3s/step - loss: 1.6970 - categorical_accuracy: 0.1961 - val_loss: 1.6080 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss improved from inf to 1.60798, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.69696-0.19608-1.60798-0.21000.h5
Epoch 2/50
21/21 [==============================] - 56s 3s/step - loss: 1.6197 - categorical_accuracy: 0.2368 - val_loss: 1.6064 - val_categorical_accuracy: 0.2800

Epoch 00002: val_loss improved from 1.60798 to 1.60641, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.61973-0.23680-1.60641-0.28000.h5
Epoch 3/50
21/21 [==============================] - 57s 3s/step - loss: 1.5788 - categorical_accuracy: 0.2866 - val_loss: 1.6019 - val_categorical_accuracy: 0.2400

Epoch 00003: val_loss improved from 1.60641 to 1.60188, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.57879-0.28658-1.60188-0.24000.h5
Epoch 4/50
21/21 [=======================

21/21 [==============================] - 54s 3s/step - loss: 1.1714 - categorical_accuracy: 0.5324 - val_loss: 1.1992 - val_categorical_accuracy: 0.5400

Epoch 00032: val_loss improved from 1.21041 to 1.19915, saving model to model_init_2021-10-2723_54_21.814102\model-00032-1.17137-0.53243-1.19915-0.54000.h5
Epoch 33/50
21/21 [==============================] - 56s 3s/step - loss: 1.1050 - categorical_accuracy: 0.5747 - val_loss: 1.1876 - val_categorical_accuracy: 0.5300

Epoch 00033: val_loss improved from 1.19915 to 1.18757, saving model to model_init_2021-10-2723_54_21.814102\model-00033-1.10495-0.57466-1.18757-0.53000.h5
Epoch 34/50
21/21 [==============================] - 56s 3s/step - loss: 1.1116 - categorical_accuracy: 0.5641 - val_loss: 1.2294 - val_categorical_accuracy: 0.5300

Epoch 00034: val_loss did not improve from 1.18757
Epoch 35/50
21/21 [==============================] - 55s 3s/step - loss: 1.1250 - categorical_accuracy: 0.5792 - val_loss: 1.1501 - val_categorical_acc

In [ ]:
# Performance of model_17 is not good enough

In [ ]:
# Trying dropouts after every conv2D layer:

In [40]:
#write your model here

model_18 = Sequential()

model_18.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))
model_18.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_18.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_18.add(TimeDistributed(BatchNormalization()))
model_18.add(Dropout(0.25))

model_18.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_18.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_18.add(TimeDistributed(BatchNormalization()))
model_18.add(Dropout(0.25))

model_18.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_18.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_18.add(TimeDistributed(BatchNormalization()))
model_18.add(Dropout(0.25))

model_18.add(TimeDistributed(Flatten()))

model_18.add(Dense(128, activation='relu'))
model_18.add(Dropout(0.5))
model_18.add(Dense(64, activation='relu'))
model_18.add(Dropout(0.5))

## using GRU as the RNN model along with softmax as our last layer.
model_18.add(GRU(128, return_sequences=False))
model_18.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [41]:
model_18.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_18.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_49 (TimeDis (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_50 (TimeDis (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_51 (TimeDis (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_52 (TimeDis (None, 20, 21, 21, 16)    64        
_________________________________________________________________
dropout_20 (Dropout)         (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_53 (TimeDis (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_54 (TimeDis (None, 20, 10, 10, 32)   

In [42]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [43]:
model_18.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 57s 3s/step - loss: 1.6948 - categorical_accuracy: 0.2202 - val_loss: 1.6121 - val_categorical_accuracy: 0.1600

Epoch 00001: val_loss improved from inf to 1.61210, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.69483-0.22021-1.61210-0.16000.h5
Epoch 2/50
21/21 [==============================] - 54s 3s/step - loss: 1.6250 - categorical_accuracy: 0.2624 - val_loss: 1.6203 - val_categorical_accuracy: 0.1600

Epoch 00002: val_loss did not improve from 1.61210
Epoch 3/50
21/21 [==============================] - 56s 3s/step - loss: 1.5976 - categorical_accuracy: 0.2670 - val_loss: 1.6297 - val_categorical_accuracy: 0.1600

Epoch 00003: val_loss did not improve from 1.61210
Epoch 4/50
21/21 [==============================] - 56s 3s/step - loss: 1.5412 - categorical_accuracy: 0.3183 - val_loss: 1.6381 - val_categorical_accuracy: 0.1700

Epoch 00004: val_loss did not improve from 1.61210
Epoch 5/50
21/21 [=================

21/21 [==============================] - 59s 3s/step - loss: 1.2375 - categorical_accuracy: 0.4555 - val_loss: 1.6486 - val_categorical_accuracy: 0.3900

Epoch 00038: val_loss did not improve from 1.59556
Epoch 39/50
21/21 [==============================] - 62s 3s/step - loss: 1.2348 - categorical_accuracy: 0.4977 - val_loss: 1.6238 - val_categorical_accuracy: 0.3800

Epoch 00039: val_loss did not improve from 1.59556
Epoch 40/50
21/21 [==============================] - 59s 3s/step - loss: 1.2412 - categorical_accuracy: 0.4992 - val_loss: 1.6311 - val_categorical_accuracy: 0.4100

Epoch 00040: val_loss did not improve from 1.59556
Epoch 41/50
21/21 [==============================] - 62s 3s/step - loss: 1.2528 - categorical_accuracy: 0.5053 - val_loss: 1.6336 - val_categorical_accuracy: 0.3600

Epoch 00041: val_loss did not improve from 1.59556
Epoch 42/50
21/21 [==============================] - 59s 3s/step - loss: 1.2369 - categorical_accuracy: 0.5189 - val_loss: 1.5608 - val_categori

In [ ]:
# Model_18 is overfitting and its overall performance is not good

In [ ]:
# Trying model 1 with 25% dropounts in 1st dense layer and 50% dropouts in 2nd dense layer:

In [44]:
#write your model here

model_19 = Sequential()

model_19.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))
model_19.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_19.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_19.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_19.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_19.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_19.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_19.add(TimeDistributed(BatchNormalization()))
model_19.add(Dropout(0.25))

model_19.add(TimeDistributed(Flatten()))

model_19.add(Dense(128, activation='relu'))
model_19.add(Dropout(0.25))
model_19.add(Dense(64, activation='relu'))
model_19.add(Dropout(0.5))

## using GRU as the RNN model along with softmax as our last layer.
model_19.add(GRU(128, return_sequences=False))
model_19.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [45]:
model_19.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_19.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_60 (TimeDis (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_61 (TimeDis (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_62 (TimeDis (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_63 (TimeDis (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_64 (TimeDis (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_65 (TimeDis (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_66 (TimeDis (None, 20, 5, 5, 64)     

In [46]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [47]:
model_19.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 61s 3s/step - loss: 1.6397 - categorical_accuracy: 0.2323 - val_loss: 1.6041 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss improved from inf to 1.60414, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.63967-0.23228-1.60414-0.21000.h5
Epoch 2/50
21/21 [==============================] - 59s 3s/step - loss: 1.5382 - categorical_accuracy: 0.2609 - val_loss: 1.6007 - val_categorical_accuracy: 0.2300

Epoch 00002: val_loss improved from 1.60414 to 1.60066, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.53824-0.26094-1.60066-0.23000.h5
Epoch 3/50
21/21 [==============================] - 57s 3s/step - loss: 1.5010 - categorical_accuracy: 0.3183 - val_loss: 1.5907 - val_categorical_accuracy: 0.2400

Epoch 00003: val_loss improved from 1.60066 to 1.59072, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.50104-0.31825-1.59072-0.24000.h5
Epoch 4/50
21/21 [=======================

21/21 [==============================] - 58s 3s/step - loss: 1.0996 - categorical_accuracy: 0.5701 - val_loss: 1.2706 - val_categorical_accuracy: 0.4900

Epoch 00029: val_loss improved from 1.28165 to 1.27063, saving model to model_init_2021-10-2723_54_21.814102\model-00029-1.09957-0.57014-1.27063-0.49000.h5
Epoch 30/50
21/21 [==============================] - 57s 3s/step - loss: 1.0951 - categorical_accuracy: 0.5777 - val_loss: 1.3144 - val_categorical_accuracy: 0.5200

Epoch 00030: val_loss did not improve from 1.27063
Epoch 31/50
21/21 [==============================] - 59s 3s/step - loss: 1.0733 - categorical_accuracy: 0.5988 - val_loss: 1.2587 - val_categorical_accuracy: 0.4600

Epoch 00031: val_loss improved from 1.27063 to 1.25872, saving model to model_init_2021-10-2723_54_21.814102\model-00031-1.07333-0.59879-1.25872-0.46000.h5
Epoch 32/50
21/21 [==============================] - 55s 3s/step - loss: 1.0864 - categorical_accuracy: 0.5732 - val_loss: 1.2573 - val_categorical_acc

In [ ]:
# Model_19 is overfitting and overall performance is not good

In [ ]:
# Trying model 1 with 50% dropounts in 1st dense layer and 25% dropouts in 2nd dense layer:

In [48]:
#write your model here

model_20 = Sequential()

model_20.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))
model_20.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_20.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_20.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_20.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_20.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_20.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_20.add(TimeDistributed(BatchNormalization()))
model_20.add(Dropout(0.25))

model_20.add(TimeDistributed(Flatten()))

model_20.add(Dense(128, activation='relu'))
model_20.add(Dropout(0.5))
model_20.add(Dense(64, activation='relu'))
model_20.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model_20.add(GRU(128, return_sequences=False))
model_20.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [49]:
model_20.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_20.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_69 (TimeDis (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_70 (TimeDis (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_71 (TimeDis (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_72 (TimeDis (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_73 (TimeDis (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_74 (TimeDis (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_75 (TimeDis (None, 20, 5, 5, 64)     

In [50]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [51]:
model_20.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 58s 3s/step - loss: 1.6632 - categorical_accuracy: 0.2428 - val_loss: 1.6013 - val_categorical_accuracy: 0.2300

Epoch 00001: val_loss improved from inf to 1.60129, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.66323-0.24284-1.60129-0.23000.h5
Epoch 2/50
21/21 [==============================] - 53s 3s/step - loss: 1.5579 - categorical_accuracy: 0.3107 - val_loss: 1.5938 - val_categorical_accuracy: 0.2600

Epoch 00002: val_loss improved from 1.60129 to 1.59379, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.55785-0.31071-1.59379-0.26000.h5
Epoch 3/50
21/21 [==============================] - 56s 3s/step - loss: 1.4994 - categorical_accuracy: 0.3484 - val_loss: 1.5890 - val_categorical_accuracy: 0.2500

Epoch 00003: val_loss improved from 1.59379 to 1.58902, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.49944-0.34842-1.58902-0.25000.h5
Epoch 4/50
21/21 [=======================

In [ ]:
# Overall performance of model_20 is not good

In [ ]:
# Using LSTM instead of GRU in model 1:

In [52]:
from tensorflow.keras.layers import LSTM

#write your model here

model_21 = Sequential()

model_21.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))
model_21.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_21.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_21.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_21.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_21.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_21.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_21.add(TimeDistributed(BatchNormalization()))
model_21.add(Dropout(0.25))

model_21.add(TimeDistributed(Flatten()))

model_21.add(Dense(128, activation='relu'))
model_21.add(Dropout(0.25))
model_21.add(Dense(64, activation='relu'))
model_21.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model_21.add(LSTM(128, return_sequences=False))
model_21.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [53]:
model_21.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_21.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_78 (TimeDis (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_79 (TimeDis (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_80 (TimeDis (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_81 (TimeDis (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_82 (TimeDis (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_83 (TimeDis (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_84 (TimeDis (None, 20, 5, 5, 64)    

In [54]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [55]:
model_21.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 59s 3s/step - loss: 1.5811 - categorical_accuracy: 0.2368 - val_loss: 1.6049 - val_categorical_accuracy: 0.2500

Epoch 00001: val_loss improved from inf to 1.60493, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.58113-0.23680-1.60493-0.25000.h5
Epoch 2/50
21/21 [==============================] - 58s 3s/step - loss: 1.4840 - categorical_accuracy: 0.3620 - val_loss: 1.6037 - val_categorical_accuracy: 0.2100

Epoch 00002: val_loss improved from 1.60493 to 1.60373, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.48399-0.36199-1.60373-0.21000.h5
Epoch 3/50
21/21 [==============================] - 57s 3s/step - loss: 1.4329 - categorical_accuracy: 0.4238 - val_loss: 1.5904 - val_categorical_accuracy: 0.2700

Epoch 00003: val_loss improved from 1.60373 to 1.59042, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.43286-0.42383-1.59042-0.27000.h5
Epoch 4/50
21/21 [=======================


Epoch 00027: val_loss did not improve from 1.17659
Epoch 28/50
21/21 [==============================] - 54s 3s/step - loss: 0.9066 - categorical_accuracy: 0.6817 - val_loss: 1.2089 - val_categorical_accuracy: 0.5200

Epoch 00028: val_loss did not improve from 1.17659
Epoch 29/50
21/21 [==============================] - 56s 3s/step - loss: 0.8773 - categorical_accuracy: 0.6863 - val_loss: 1.2068 - val_categorical_accuracy: 0.5500

Epoch 00029: val_loss did not improve from 1.17659
Epoch 30/50
21/21 [==============================] - 56s 3s/step - loss: 0.8428 - categorical_accuracy: 0.6878 - val_loss: 1.1850 - val_categorical_accuracy: 0.5600

Epoch 00030: val_loss did not improve from 1.17659
Epoch 31/50
21/21 [==============================] - 55s 3s/step - loss: 0.8625 - categorical_accuracy: 0.6968 - val_loss: 1.2071 - val_categorical_accuracy: 0.5200

Epoch 00031: val_loss did not improve from 1.17659
Epoch 32/50
21/21 [==============================] - 55s 3s/step - loss: 0.8314 

In [ ]:
# Model_21 is overfitting

In [ ]:
# Trying GRU before the dense layers in model 1:

In [56]:
#write your model here

model_22 = Sequential()

model_22.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))
model_22.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_22.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_22.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_22.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_22.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_22.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_22.add(TimeDistributed(BatchNormalization()))
model_22.add(Dropout(0.25))

model_22.add(TimeDistributed(Flatten()))

## using GRU as the RNN model along with softmax as our last layer.
model_22.add(GRU(128, return_sequences=False))

model_22.add(Dense(128, activation='relu'))
model_22.add(Dropout(0.25))
model_22.add(Dense(64, activation='relu'))
model_22.add(Dropout(0.25))

model_22.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [57]:
model_22.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_22.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_87 (TimeDis (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_88 (TimeDis (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_89 (TimeDis (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_90 (TimeDis (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_91 (TimeDis (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_92 (TimeDis (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_93 (TimeDis (None, 20, 5, 5, 64)    

In [58]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [59]:
model_22.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 57s 3s/step - loss: 1.5822 - categorical_accuracy: 0.2549 - val_loss: 1.5982 - val_categorical_accuracy: 0.2200

Epoch 00001: val_loss improved from inf to 1.59823, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.58216-0.25490-1.59823-0.22000.h5
Epoch 2/50
21/21 [==============================] - 53s 3s/step - loss: 1.4932 - categorical_accuracy: 0.3424 - val_loss: 1.5996 - val_categorical_accuracy: 0.2500

Epoch 00002: val_loss did not improve from 1.59823
Epoch 3/50
21/21 [==============================] - 57s 3s/step - loss: 1.4547 - categorical_accuracy: 0.3590 - val_loss: 1.5924 - val_categorical_accuracy: 0.2800

Epoch 00003: val_loss improved from 1.59823 to 1.59238, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.45468-0.35897-1.59238-0.28000.h5
Epoch 4/50
21/21 [==============================] - 55s 3s/step - loss: 1.4135 - categorical_accuracy: 0.3891 - val_loss: 1.5768 - val_categorical


Epoch 00029: val_loss improved from 1.22834 to 1.20885, saving model to model_init_2021-10-2723_54_21.814102\model-00029-1.06355-0.57466-1.20885-0.56000.h5
Epoch 30/50
21/21 [==============================] - 57s 3s/step - loss: 1.0509 - categorical_accuracy: 0.5656 - val_loss: 1.2238 - val_categorical_accuracy: 0.5400

Epoch 00030: val_loss did not improve from 1.20885
Epoch 31/50
21/21 [==============================] - 58s 3s/step - loss: 1.0237 - categorical_accuracy: 0.6244 - val_loss: 1.2235 - val_categorical_accuracy: 0.5100

Epoch 00031: val_loss did not improve from 1.20885
Epoch 32/50
21/21 [==============================] - 59s 3s/step - loss: 1.0657 - categorical_accuracy: 0.5928 - val_loss: 1.2345 - val_categorical_accuracy: 0.5100

Epoch 00032: val_loss did not improve from 1.20885
Epoch 33/50
21/21 [==============================] - 58s 3s/step - loss: 1.0178 - categorical_accuracy: 0.6124 - val_loss: 1.1823 - val_categorical_accuracy: 0.5000

Epoch 00033: val_loss impr

In [ ]:
# model_22 is overfitting and does not have a good performance

In [ ]:
# Trying LSTM in model 10 instead of GRU:

In [60]:
#write your model here

model_23 = Sequential()

model_23.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))
model_23.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_23.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_23.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_23.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_23.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_23.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_23.add(TimeDistributed(BatchNormalization()))
model_23.add(Dropout(0.25))

model_23.add(TimeDistributed(Flatten()))

## using LSTM as the RNN model along with softmax as our last layer.
model_23.add(LSTM(128, return_sequences=False))

model_23.add(Dense(128, activation='relu'))
model_23.add(Dropout(0.25))
model_23.add(Dense(64, activation='relu'))
model_23.add(Dropout(0.25))

model_23.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [61]:
model_23.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_23.summary())

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_96 (TimeDis (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_97 (TimeDis (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_98 (TimeDis (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_99 (TimeDis (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_100 (TimeDi (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_101 (TimeDi (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_102 (TimeDi (None, 20, 5, 5, 64)    

In [62]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [63]:
model_23.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 57s 3s/step - loss: 1.6027 - categorical_accuracy: 0.2700 - val_loss: 1.6057 - val_categorical_accuracy: 0.2400

Epoch 00001: val_loss improved from inf to 1.60574, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.60268-0.26998-1.60574-0.24000.h5
Epoch 2/50
21/21 [==============================] - 57s 3s/step - loss: 1.5338 - categorical_accuracy: 0.3167 - val_loss: 1.5951 - val_categorical_accuracy: 0.2500

Epoch 00002: val_loss improved from 1.60574 to 1.59514, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.53380-0.31674-1.59514-0.25000.h5
Epoch 3/50
21/21 [==============================] - 57s 3s/step - loss: 1.4887 - categorical_accuracy: 0.3379 - val_loss: 1.6045 - val_categorical_accuracy: 0.2300

Epoch 00003: val_loss did not improve from 1.59514
Epoch 4/50
21/21 [==============================] - 55s 3s/step - loss: 1.4375 - categorical_accuracy: 0.3967 - val_loss: 1.5951 - val_categorical

In [ ]:
# model_23 is overfitting

In [ ]:
# Removing the dropouts after the third Conv2D layer and trying again:

In [64]:
#write your model here

model_24 = Sequential()

model_24.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))
model_24.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_24.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_24.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_24.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_24.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_24.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model_24.add(TimeDistributed(BatchNormalization()))

model_24.add(TimeDistributed(Flatten()))

## using LSTM as the RNN model along with softmax as our last layer.
model_24.add(LSTM(128, return_sequences=False))

model_24.add(Dense(128, activation='relu'))
model_24.add(Dropout(0.25))
model_24.add(Dense(64, activation='relu'))
model_24.add(Dropout(0.25))

model_24.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [65]:
model_24.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_24.summary())

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_105 (TimeDi (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_106 (TimeDi (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_107 (TimeDi (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_108 (TimeDi (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_109 (TimeDi (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_110 (TimeDi (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_111 (TimeDi (None, 20, 5, 5, 64)    

In [66]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [67]:
model_24.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 60s 3s/step - loss: 1.6025 - categorical_accuracy: 0.2353 - val_loss: 1.6057 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss improved from inf to 1.60567, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.60247-0.23529-1.60567-0.21000.h5
Epoch 2/50
21/21 [==============================] - 56s 3s/step - loss: 1.5152 - categorical_accuracy: 0.3454 - val_loss: 1.6065 - val_categorical_accuracy: 0.1800

Epoch 00002: val_loss did not improve from 1.60567
Epoch 3/50
21/21 [==============================] - 57s 3s/step - loss: 1.5001 - categorical_accuracy: 0.3424 - val_loss: 1.6026 - val_categorical_accuracy: 0.1800

Epoch 00003: val_loss improved from 1.60567 to 1.60260, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.50015-0.34238-1.60260-0.18000.h5
Epoch 4/50
21/21 [==============================] - 58s 3s/step - loss: 1.4461 - categorical_accuracy: 0.3982 - val_loss: 1.5945 - val_categorical


Epoch 00028: val_loss did not improve from 1.31032
Epoch 29/50
21/21 [==============================] - 56s 3s/step - loss: 0.9385 - categorical_accuracy: 0.7134 - val_loss: 1.1858 - val_categorical_accuracy: 0.5700

Epoch 00029: val_loss improved from 1.31032 to 1.18580, saving model to model_init_2021-10-2723_54_21.814102\model-00029-0.93848-0.71342-1.18580-0.57000.h5
Epoch 30/50
21/21 [==============================] - 55s 3s/step - loss: 0.9642 - categorical_accuracy: 0.6682 - val_loss: 1.2924 - val_categorical_accuracy: 0.5300

Epoch 00030: val_loss did not improve from 1.18580
Epoch 31/50
21/21 [==============================] - 57s 3s/step - loss: 0.9106 - categorical_accuracy: 0.7255 - val_loss: 1.2810 - val_categorical_accuracy: 0.5300

Epoch 00031: val_loss did not improve from 1.18580
Epoch 32/50
21/21 [==============================] - 56s 3s/step - loss: 0.9211 - categorical_accuracy: 0.7134 - val_loss: 1.2903 - val_categorical_accuracy: 0.5100

Epoch 00032: val_loss did 

In [ ]:
# Model_24 is overfitting. Also, as far as validation accuracy is concerned, there was no benefit of removing the dropouts
# after the third Conv2D layer. In fact, doing so reduced the validation accuracy as compared to epoch 50 of model_23

In [ ]:
# Running model 1 with only 1 dense(128) layer and 25% dropouts after that dense layer:

In [68]:
#write your model here

model_25 = Sequential()

model_25.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))


model_25.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_25.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_25.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_25.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_25.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_25.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_25.add(TimeDistributed(BatchNormalization()))
model_25.add(Dropout(0.25))

model_25.add(TimeDistributed(Flatten()))

model_25.add(Dense(128, activation='relu'))
model_25.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model_25.add(GRU(128, return_sequences=False))
model_25.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [69]:
model_25.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_25.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_114 (TimeDi (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_115 (TimeDi (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_116 (TimeDi (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_117 (TimeDi (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_118 (TimeDi (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_119 (TimeDi (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_120 (TimeDi (None, 20, 5, 5, 64)    

In [70]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [71]:
model_25.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 59s 3s/step - loss: 1.5903 - categorical_accuracy: 0.2730 - val_loss: 1.6017 - val_categorical_accuracy: 0.2300

Epoch 00001: val_loss improved from inf to 1.60168, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.59034-0.27300-1.60168-0.23000.h5
Epoch 2/50
21/21 [==============================] - 56s 3s/step - loss: 1.4488 - categorical_accuracy: 0.3771 - val_loss: 1.6003 - val_categorical_accuracy: 0.1800

Epoch 00002: val_loss improved from 1.60168 to 1.60025, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.44880-0.37707-1.60025-0.18000.h5
Epoch 3/50
21/21 [==============================] - 57s 3s/step - loss: 1.3638 - categorical_accuracy: 0.4495 - val_loss: 1.5760 - val_categorical_accuracy: 0.2500

Epoch 00003: val_loss improved from 1.60025 to 1.57595, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.36381-0.44947-1.57595-0.25000.h5
Epoch 4/50
21/21 [=======================

In [ ]:
# model_25 is overfitting a lot. Moreover, the validation accuracy is poor.

In [ ]:
# Running model 1 with only 1 dense(128) layer and 50% dropout:

In [73]:
#write your model here

model_26 = Sequential()

model_26.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))

model_26.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_26.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_26.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_26.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_26.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_26.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_26.add(TimeDistributed(BatchNormalization()))
model_26.add(Dropout(0.25))

model_26.add(TimeDistributed(Flatten()))

model_26.add(Dense(128, activation='relu'))
model_26.add(Dropout(0.50))

## using GRU as the RNN model along with softmax as our last layer.
model_26.add(GRU(128, return_sequences=False))
model_26.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [74]:
model_26.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_26.summary())

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_132 (TimeDi (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_133 (TimeDi (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_134 (TimeDi (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_135 (TimeDi (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_136 (TimeDi (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_137 (TimeDi (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_138 (TimeDi (None, 20, 5, 5, 64)    

In [75]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [76]:
model_26.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 59s 3s/step - loss: 1.5907 - categorical_accuracy: 0.2896 - val_loss: 1.6180 - val_categorical_accuracy: 0.1900

Epoch 00001: val_loss improved from inf to 1.61796, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.59073-0.28959-1.61796-0.19000.h5
Epoch 2/50
21/21 [==============================] - 58s 3s/step - loss: 1.4987 - categorical_accuracy: 0.3152 - val_loss: 1.6015 - val_categorical_accuracy: 0.2200

Epoch 00002: val_loss improved from 1.61796 to 1.60149, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.49867-0.31523-1.60149-0.22000.h5
Epoch 3/50
21/21 [==============================] - 62s 3s/step - loss: 1.4109 - categorical_accuracy: 0.4193 - val_loss: 1.5987 - val_categorical_accuracy: 0.2200

Epoch 00003: val_loss improved from 1.60149 to 1.59874, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.41090-0.41931-1.59874-0.22000.h5
Epoch 4/50
21/21 [=======================


Epoch 00029: val_loss did not improve from 1.27806
Epoch 30/50
21/21 [==============================] - 59s 3s/step - loss: 1.0293 - categorical_accuracy: 0.5973 - val_loss: 1.2588 - val_categorical_accuracy: 0.5000

Epoch 00030: val_loss improved from 1.27806 to 1.25881, saving model to model_init_2021-10-2723_54_21.814102\model-00030-1.02928-0.59729-1.25881-0.50000.h5
Epoch 31/50
21/21 [==============================] - 64s 3s/step - loss: 1.0028 - categorical_accuracy: 0.6169 - val_loss: 1.2622 - val_categorical_accuracy: 0.5100

Epoch 00031: val_loss did not improve from 1.25881
Epoch 32/50
21/21 [==============================] - 63s 3s/step - loss: 0.9964 - categorical_accuracy: 0.6199 - val_loss: 1.1936 - val_categorical_accuracy: 0.5800

Epoch 00032: val_loss improved from 1.25881 to 1.19356, saving model to model_init_2021-10-2723_54_21.814102\model-00032-0.99639-0.61991-1.19356-0.58000.h5
Epoch 33/50
21/21 [==============================] - 63s 3s/step - loss: 0.9718 - categ

In [ ]:
# Epoch 49 gave good result for model_26. Increasing the dropouts from 25% in model_25 to 50% in model_26 in the dense
# layer improved the validation accuracy.

In [ ]:
# Running model 1 with only 1 dense(64) layer and 25% dropout:

In [77]:
#write your model here

model_27 = Sequential()

model_27.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))

model_27.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_27.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_27.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_27.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_27.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_27.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_27.add(TimeDistributed(BatchNormalization()))
model_27.add(Dropout(0.25))

model_27.add(TimeDistributed(Flatten()))

model_27.add(Dense(64, activation='relu'))
model_27.add(Dropout(0.25))

## using GRU as the RNN model along with softmax as our last layer.
model_27.add(GRU(128, return_sequences=False))
model_27.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [78]:
model_27.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_27.summary())

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_141 (TimeDi (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_142 (TimeDi (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_143 (TimeDi (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_144 (TimeDi (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_145 (TimeDi (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_146 (TimeDi (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_147 (TimeDi (None, 20, 5, 5, 64)    

In [79]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [80]:
model_27.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 60s 3s/step - loss: 1.5928 - categorical_accuracy: 0.2655 - val_loss: 1.6034 - val_categorical_accuracy: 0.2300

Epoch 00001: val_loss improved from inf to 1.60342, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.59277-0.26546-1.60342-0.23000.h5
Epoch 2/50
21/21 [==============================] - 56s 3s/step - loss: 1.5020 - categorical_accuracy: 0.3213 - val_loss: 1.5915 - val_categorical_accuracy: 0.2800

Epoch 00002: val_loss improved from 1.60342 to 1.59147, saving model to model_init_2021-10-2723_54_21.814102\model-00002-1.50200-0.32127-1.59147-0.28000.h5
Epoch 3/50
21/21 [==============================] - 59s 3s/step - loss: 1.4533 - categorical_accuracy: 0.3846 - val_loss: 1.5947 - val_categorical_accuracy: 0.2100

Epoch 00003: val_loss did not improve from 1.59147
Epoch 4/50
21/21 [==============================] - 55s 3s/step - loss: 1.3768 - categorical_accuracy: 0.4404 - val_loss: 1.5880 - val_categorical


Epoch 00028: val_loss did not improve from 1.26408
Epoch 29/50
21/21 [==============================] - 57s 3s/step - loss: 1.0323 - categorical_accuracy: 0.6244 - val_loss: 1.2808 - val_categorical_accuracy: 0.5000

Epoch 00029: val_loss did not improve from 1.26408
Epoch 30/50
21/21 [==============================] - 57s 3s/step - loss: 1.0306 - categorical_accuracy: 0.6139 - val_loss: 1.3096 - val_categorical_accuracy: 0.4900

Epoch 00030: val_loss did not improve from 1.26408
Epoch 31/50
21/21 [==============================] - 57s 3s/step - loss: 1.0205 - categorical_accuracy: 0.6395 - val_loss: 1.2497 - val_categorical_accuracy: 0.5100

Epoch 00031: val_loss improved from 1.26408 to 1.24968, saving model to model_init_2021-10-2723_54_21.814102\model-00031-1.02050-0.63952-1.24968-0.51000.h5
Epoch 32/50
21/21 [==============================] - 54s 3s/step - loss: 0.9940 - categorical_accuracy: 0.6365 - val_loss: 1.2688 - val_categorical_accuracy: 0.5100

Epoch 00032: val_loss did 

In [ ]:
# model_27 is overfitting. When we compare it with model_25, in the 50th epoch, the categorical accuracy has reduced
# (from 77.83% to 70.14%), while the validation accuracy has increased (from 45 to 58).

In [ ]:
# Running model 1 with only 1 dense(64) layer and 50% dropout:

In [81]:
#write your model here

model_28 = Sequential()

model_28.add(TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=(20,84,84,3)))

model_28.add(TimeDistributed(Conv2D(16, (3,3),padding='same', activation='relu')))
model_28.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_28.add(TimeDistributed(Conv2D(32, (3,3),padding='same', activation='relu')))
model_28.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_28.add(TimeDistributed(Conv2D(64, (2,2),padding='same', activation='relu')))
model_28.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model_28.add(TimeDistributed(BatchNormalization()))
model_28.add(Dropout(0.25))

model_28.add(TimeDistributed(Flatten()))

model_28.add(Dense(64, activation='relu'))
model_28.add(Dropout(0.50))

## using GRU as the RNN model along with softmax as our last layer.
model_28.add(GRU(64, return_sequences=False))
model_28.add(Dense(5, activation='softmax')) # using Softmax as last layer

In [82]:
model_28.compile(optimizer=optimiser_2, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_28.summary())

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_150 (TimeDi (None, 20, 42, 42, 8)     224       
_________________________________________________________________
time_distributed_151 (TimeDi (None, 20, 42, 42, 16)    1168      
_________________________________________________________________
time_distributed_152 (TimeDi (None, 20, 21, 21, 16)    0         
_________________________________________________________________
time_distributed_153 (TimeDi (None, 20, 21, 21, 32)    4640      
_________________________________________________________________
time_distributed_154 (TimeDi (None, 20, 10, 10, 32)    0         
_________________________________________________________________
time_distributed_155 (TimeDi (None, 20, 10, 10, 64)    8256      
_________________________________________________________________
time_distributed_156 (TimeDi (None, 20, 5, 5, 64)    

In [83]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [84]:
model_28.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
21/21 [==============================] - 59s 3s/step - loss: 1.5981 - categorical_accuracy: 0.2504 - val_loss: 1.5980 - val_categorical_accuracy: 0.2600

Epoch 00001: val_loss improved from inf to 1.59802, saving model to model_init_2021-10-2723_54_21.814102\model-00001-1.59807-0.25038-1.59802-0.26000.h5
Epoch 2/50
21/21 [==============================] - 56s 3s/step - loss: 1.5196 - categorical_accuracy: 0.3288 - val_loss: 1.5994 - val_categorical_accuracy: 0.2400

Epoch 00002: val_loss did not improve from 1.59802
Epoch 3/50
21/21 [==============================] - 58s 3s/step - loss: 1.4420 - categorical_accuracy: 0.3816 - val_loss: 1.5948 - val_categorical_accuracy: 0.2500

Epoch 00003: val_loss improved from 1.59802 to 1.59481, saving model to model_init_2021-10-2723_54_21.814102\model-00003-1.44202-0.38160-1.59481-0.25000.h5
Epoch 4/50
21/21 [==============================] - 57s 3s/step - loss: 1.4229 - categorical_accuracy: 0.3906 - val_loss: 1.5982 - val_categorical

Epoch 30/50
21/21 [==============================] - 57s 3s/step - loss: 1.1496 - categorical_accuracy: 0.5520 - val_loss: 1.3295 - val_categorical_accuracy: 0.4800

Epoch 00030: val_loss did not improve from 1.23756
Epoch 31/50
21/21 [==============================] - 57s 3s/step - loss: 1.1461 - categorical_accuracy: 0.5551 - val_loss: 1.3173 - val_categorical_accuracy: 0.5200

Epoch 00031: val_loss did not improve from 1.23756
Epoch 32/50
21/21 [==============================] - 56s 3s/step - loss: 1.1513 - categorical_accuracy: 0.5354 - val_loss: 1.2744 - val_categorical_accuracy: 0.4700

Epoch 00032: val_loss did not improve from 1.23756
Epoch 33/50
21/21 [==============================] - 57s 3s/step - loss: 1.1048 - categorical_accuracy: 0.5897 - val_loss: 1.2945 - val_categorical_accuracy: 0.5300

Epoch 00033: val_loss did not improve from 1.23756
Epoch 34/50
21/21 [==============================] - 54s 3s/step - loss: 1.1123 - categorical_accuracy: 0.5626 - val_loss: 1.2945 - 

In [ ]:
# Overall performance of model_28 is not good. When we compare this with model_27, we notice that both categorical accuracy
# (from 70.14% to 60.18%) and validation accuracy (58% to 54%) have gone down.

# CNN - RNN Models (With Transfer Learning)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
# source_path = train_path i.e. path of 663 folders
# folder_list = train_doc i.e. csv file
# batch_size = 64

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    # It is not possible to work with all the 30 images, as it will take too long processing time.
    # So lets choose randomly 15 images, as this is more computationally expensive
    img_idx = [1,2,4,6,10,12,14,16,18,20,22,24,27,28,29] #create a list of image numbers you want to use for a particular video(incase if u want to try with lesser images)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,15,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image[:,20:140,:],(84,84)).astype(np.float32)
                    normalizedImg = image/255.0                    
                    
                    batch_data[folder,idx,:,:,0] = (normalizedImg[:,:,0]) #normalise and feed in the image # divide by 255.0
                    batch_data[folder,idx,:,:,1] = (normalizedImg[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (normalizedImg[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(len(t)%batch_size)!=0:
            batch_data = np.zeros((len(t)%batch_size,15,84,84,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image[:,20:140,:],(84,84)).astype(np.float32)
                    normalizedImg = image/255.0                    
                    
                    batch_data[folder,idx,:,:,0] = (normalizedImg[:,:,0]) #normalise and feed in the image # divide by 255.0
                    batch_data[folder,idx,:,:,1] = (normalizedImg[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (normalizedImg[:,:,2]) #normalise and feed in the imagee
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1 # OHE
            yield batch_data, batch_labels

In [6]:
num_epochs = 30

In [7]:
batch_size = 64

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from tensorflow.keras.layers import LSTM

from keras.applications.vgg16 import VGG16

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(84,84,3))
x = base_model.output
x = Flatten()(x)
# x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs= base_model.input, outputs=features)

for layer in base_model.layers:
    layer.trainable = False
    
model_29 = Sequential()
model_29.add(TimeDistributed(conv_model, input_shape=(15,84,84,3)))
model_29.add(LSTM(32, return_sequences=True))
model_29.add(LSTM(16))
model_29.add(Dropout(0.25))
model_29.add(Dense(8, activation='relu'))
model_29.add(Dense(5, activation='softmax'))

In [19]:
from keras.optimizers import adam_v2

optimiser_3 =adam_v2.Adam(0.001) #write your optimizer

model_29.compile(optimizer=optimiser_3, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_29.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_2 (TimeDist (None, 15, 64)            14845824  
_________________________________________________________________
lstm_2 (LSTM)                (None, 15, 32)            12416     
_________________________________________________________________
lstm_3 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 45        
Total params: 14,861,557
Trainable params: 146,869
Non-trainable params: 14,714,688
____________________________________

In [10]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [20]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [12]:
# if #images=55, batch_size=5, then we have 11 batches and 11 epochs
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
# if #images=55, batch_size=5, then we have 11 batches and 3 pending images, therefore 12 epoch will be reqd for last 3 images
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [21]:
model_29.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
11/11 [==============================] - 45s 4s/step - loss: 1.6053 - categorical_accuracy: 0.2066 - val_loss: 1.5933 - val_categorical_accuracy: 0.3000

Epoch 00001: val_loss improved from inf to 1.59328, saving model to model_init_2021-10-2820_51_26.241077\model-00001-1.60527-0.20664-1.59328-0.30000.h5
Epoch 2/30
11/11 [==============================] - 42s 4s/step - loss: 1.5501 - categorical_accuracy: 0.3198 - val_loss: 1.5667 - val_categorical_accuracy: 0.2800

Epoch 00002: val_loss improved from 1.59328 to 1.56668, saving model to model_init_2021-10-2820_51_26.241077\model-00002-1.55006-0.31976-1.56668-0.28000.h5
Epoch 3/30
11/11 [==============================] - 44s 4s/step - loss: 1.5042 - categorical_accuracy: 0.3680 - val_loss: 1.4917 - val_categorical_accuracy: 0.3900

Epoch 00003: val_loss improved from 1.56668 to 1.49169, saving model to model_init_2021-10-2820_51_26.241077\model-00003-1.50416-0.36802-1.49169-0.39000.h5
Epoch 4/30
11/11 [=======================

11/11 [==============================] - 42s 4s/step - loss: 0.4814 - categorical_accuracy: 0.8673 - val_loss: 1.3148 - val_categorical_accuracy: 0.5900

Epoch 00030: val_loss did not improve from 1.15633


In [ ]:
# Model_29 is overfitting

In [ ]:
# Trying with GRU(64) and GRU(32):

In [14]:
#write your model here

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(84,84,3))
x = base_model.output
x = Flatten()(x)
# x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs= base_model.input, outputs=features)

for layer in base_model.layers:
    layer.trainable = False
    
model_30 = Sequential()
model_30.add(TimeDistributed(conv_model, input_shape=(15,84,84,3)))
model_30.add(GRU(64, return_sequences=True))
model_30.add(GRU(32))
model_30.add(Dropout(0.25))
model_30.add(Dense(8, activation='relu'))
model_30.add(Dense(5, activation='softmax'))

In [15]:
model_30.compile(optimizer=optimiser_3, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_30.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 15, 64)            14845824  
_________________________________________________________________
gru (GRU)                    (None, 15, 64)            24960     
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                9408      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 45        
Total params: 14,880,501
Trainable params: 165,813
Non-trainable params: 14,714,688
____________________________________

In [16]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [17]:
model_30.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
11/11 [==============================] - 48s 5s/step - loss: 1.6149 - categorical_accuracy: 0.1991 - val_loss: 1.5600 - val_categorical_accuracy: 0.2600

Epoch 00001: val_loss improved from inf to 1.55999, saving model to model_init_2021-10-2820_51_26.241077\model-00001-1.61490-0.19910-1.55999-0.26000.h5
Epoch 2/30
11/11 [==============================] - 42s 4s/step - loss: 1.5699 - categorical_accuracy: 0.2730 - val_loss: 1.4788 - val_categorical_accuracy: 0.3100

Epoch 00002: val_loss improved from 1.55999 to 1.47879, saving model to model_init_2021-10-2820_51_26.241077\model-00002-1.56994-0.27300-1.47879-0.31000.h5
Epoch 3/30
11/11 [==============================] - 45s 4s/step - loss: 1.4425 - categorical_accuracy: 0.3499 - val_loss: 1.4997 - val_categorical_accuracy: 0.3100

Epoch 00003: val_loss did not improve from 1.47879
Epoch 4/30
11/11 [==============================] - 43s 4s/step - loss: 1.3353 - categorical_accuracy: 0.4057 - val_loss: 1.4126 - val_categorical

In [ ]:
# model_30 is massively overfitting

In [ ]:
# Trying with GRU(64) with return sequences = False:

In [22]:
#write your model here
from keras.applications.vgg16 import VGG16

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(84,84,3))
x = base_model.output
x = Flatten()(x)
# x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs= base_model.input, outputs=features)

for layer in base_model.layers:
    layer.trainable = False
    
model_31 = Sequential()
model_31.add(TimeDistributed(conv_model, input_shape=(15,84,84,3)))
model_31.add(GRU(64))
model_31.add(Dropout(0.25))
model_31.add(Dense(8, activation='relu'))
model_31.add(Dense(5, activation='softmax'))

In [23]:
model_31.compile(optimizer=optimiser_3, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_31.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_3 (TimeDist (None, 15, 64)            14845824  
_________________________________________________________________
gru_2 (GRU)                  (None, 64)                24960     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 520       
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 45        
Total params: 14,871,349
Trainable params: 156,661
Non-trainable params: 14,714,688
_________________________________________________________________
None


In [24]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [25]:
model_31.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
11/11 [==============================] - 45s 4s/step - loss: 1.6852 - categorical_accuracy: 0.1825 - val_loss: 1.6281 - val_categorical_accuracy: 0.1700

Epoch 00001: val_loss improved from inf to 1.62811, saving model to model_init_2021-10-2820_51_26.241077\model-00001-1.68517-0.18250-1.62811-0.17000.h5
Epoch 2/30
11/11 [==============================] - 41s 4s/step - loss: 1.6406 - categorical_accuracy: 0.1916 - val_loss: 1.6096 - val_categorical_accuracy: 0.1500

Epoch 00002: val_loss improved from 1.62811 to 1.60964, saving model to model_init_2021-10-2820_51_26.241077\model-00002-1.64061-0.19155-1.60964-0.15000.h5
Epoch 3/30
11/11 [==============================] - 43s 4s/step - loss: 1.6222 - categorical_accuracy: 0.1870 - val_loss: 1.6108 - val_categorical_accuracy: 0.1700

Epoch 00003: val_loss did not improve from 1.60964
Epoch 4/30
11/11 [==============================] - 41s 4s/step - loss: 1.6075 - categorical_accuracy: 0.1946 - val_loss: 1.6129 - val_categorical

In [ ]:
# Overall performance of model_31 is not good

In [ ]:
# Trying with LSTM 16 and 8:

In [26]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from tensorflow.keras.layers import LSTM

#write your model here
from keras.applications.vgg16 import VGG16

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(84,84,3))
x = base_model.output
x = Flatten()(x)
# x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs= base_model.input, outputs=features)

for layer in base_model.layers:
    layer.trainable = False
    
model_32 = Sequential()
model_32.add(TimeDistributed(conv_model, input_shape=(15,84,84,3)))
model_32.add(LSTM(16, return_sequences=True))
model_32.add(LSTM(8))
model_32.add(Dropout(0.25))
model_32.add(Dense(8, activation='relu'))
model_32.add(Dense(5, activation='softmax'))

In [27]:
model_32.compile(optimizer=optimiser_3, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_32.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_4 (TimeDist (None, 15, 64)            14845824  
_________________________________________________________________
lstm_4 (LSTM)                (None, 15, 16)            5184      
_________________________________________________________________
lstm_5 (LSTM)                (None, 8)                 800       
_________________________________________________________________
dropout_4 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_14 (Dense)             (None, 5)                 45        
Total params: 14,851,925
Trainable params: 137,237
Non-trainable params: 14,714,688
____________________________________

In [28]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [29]:
model_32.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
11/11 [==============================] - 46s 4s/step - loss: 1.6081 - categorical_accuracy: 0.2021 - val_loss: 1.5977 - val_categorical_accuracy: 0.2300

Epoch 00001: val_loss improved from inf to 1.59768, saving model to model_init_2021-10-2820_51_26.241077\model-00001-1.60807-0.20211-1.59768-0.23000.h5
Epoch 2/30
11/11 [==============================] - 47s 5s/step - loss: 1.6026 - categorical_accuracy: 0.2278 - val_loss: 1.6025 - val_categorical_accuracy: 0.1900

Epoch 00002: val_loss did not improve from 1.59768
Epoch 3/30
11/11 [==============================] - 49s 5s/step - loss: 1.5930 - categorical_accuracy: 0.2021 - val_loss: 1.5905 - val_categorical_accuracy: 0.2200

Epoch 00003: val_loss improved from 1.59768 to 1.59050, saving model to model_init_2021-10-2820_51_26.241077\model-00003-1.59295-0.20211-1.59050-0.22000.h5
Epoch 4/30
11/11 [==============================] - 43s 4s/step - loss: 1.5946 - categorical_accuracy: 0.2217 - val_loss: 1.5816 - val_categorical


Epoch 00030: val_loss did not improve from 1.50086


In [ ]:
# Performance of model_32 is too poor

In [ ]:
# Trying to train the weights in model 1:

In [9]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from tensorflow.keras.layers import LSTM

from keras.applications.vgg16 import VGG16

#write your model here
from keras.applications.vgg16 import VGG16

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(84,84,3))
x = base_model.output
x = Flatten()(x)
# x = Dropout(0.1)(x)
# x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs= base_model.input, outputs=features)

for layer in base_model.layers:
    layer.trainable = True
    
model_33 = Sequential()
model_33.add(TimeDistributed(conv_model, input_shape=(15,84,84,3)))
model_33.add(LSTM(32, return_sequences=True))
model_33.add(LSTM(16))
model_33.add(Dropout(0.25))
model_33.add(Dense(8, activation='relu'))
model_33.add(Dense(5, activation='softmax'))

In [10]:
from keras.optimizers import adam_v2

optimiser_3 =adam_v2.Adam(0.001) #write your optimizer

model_33.compile(optimizer=optimiser_3, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_33.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 15, 64)            14845824  
_________________________________________________________________
lstm_2 (LSTM)                (None, 15, 32)            12416     
_________________________________________________________________
lstm_3 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 45        
Total params: 14,861,557
Trainable params: 14,861,557
Non-trainable params: 0
__________________________________________

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [13]:
# if #images=55, batch_size=5, then we have 11 batches and 11 epochs
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
# if #images=55, batch_size=5, then we have 11 batches and 3 pending images, therefore 12 epoch will be reqd for last 3 images
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
model_33.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:\Users\Amit\Documents\Academics\upGrad\Deep Learning\Project_data\train ; batch size = 64
Epoch 1/30
11/11 [==============================] - 69s 5s/step - loss: 1.6407 - categorical_accuracy: 0.1704 - val_loss: 1.6208 - val_categorical_accuracy: 0.2100

Epoch 00001: val_loss improved from inf to 1.62085, saving model to model_init_2021-10-2823_31_32.634291\model-00001-1.64069-0.17044-1.62085-0.21000.h5
Epoch 2/30
11/11 [==============================] - 42s 4s/step - loss: 1.6435 - categorical_accuracy: 0.1795 - val_loss: 1.6511 - val_categorical_accuracy: 0.1300

Epoch 00002: val_loss did not improve from 1.62085
Epoch 3/30
11/11 [==============================] - 43s 4s/step - loss: 1.6167 - categorical_accuracy: 0.2097 - val_loss: 1.6158 - val_categorical_accuracy: 0.1800

Epoch 00003: val_loss improved from 1.62085 to 1.61580, saving model to model_init_2021-10-2823_31_32.634291\model-00003-1.61674-0.20965-1.61580-0.18000.h5
Epoch 4/30
11/11 [=====================